In [78]:
import pandas as pd
import numpy as np
import nltk
import string
import re
import os
import re
import time
import csv
import random
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import date, timedelta, datetime
from urllib.request import urlopen
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
#from fake_useragent import UserAgent
import random
from time import sleep
from bs4 import BeautifulSoup

In [79]:

import requests

api_keys = [
    'ebc4f1b043msh8cd86a1b34ccf04p1a2452jsn26419110e8fb',
    "bc0a45feabmsha0087ca72d8d32cp1a86cfjsn98c21a8c343e",
    "f8845247dbmsh947bb77edae70bep17a8edjsn8e4804d7cfb5"
]

In [83]:
with open("/Users/kseniayakunina/se/Диплом/urls_snp.txt", "r", encoding="utf-8") as f:
    urls = f.read().split("\n")

In [84]:
len(urls)

1994

In [85]:
ticker_data = []
company_data = []
title_data = []
date_data = []
time_data = []
quarter_data = []
company_info_table = []
Speech_table = []
QA_table = []
Comp_part_table = []
speech = []
speaker = []
QA_speech = []
QA_speaker = []
comp_parts = []
comp_pos = []
conf_parts = []
conf_pos = []
Conf_part_table = []
comp_ccid = []
speech_ccid = []
QA_ccid = []
comppart_ccid = []
confpart_ccid = []
speech_scriptid = []
QA_scriptid = []
comppart_partid = []
confpart_partid = []
test_list = []
full_text = []
ticker_list = []
counter = 0
counter2 = 0
counter3 = 0

try:
    counter = company_info_table['CC ID'].iloc[-1] + 1
    counter2 = QA_table['Script ID'].iloc[-1] + 1
    counter3 = Conf_part_table['Part ID'].iloc[-1] + 1
except:
    counter = 0
    counter2 = 0
    counter3 = 0
for _, i in enumerate(urls):
    article_id = i.replace("https://seekingalpha.com/article/", "")[:7]
    retry = 3
    while retry > 0:
        # response = requests.get(
        #     f"https://seekingalpha.com/api/v3/articles/{article_id}?include=author%2CprimaryTickers%2CsecondaryTickers%2CotherTags%2Cpresentations%2Cpresentations.slides%2Cauthor.authorResearch%2Cco_authors%2CpromotedService%2Csentiments",
        #     headers={
        #         "User-Agent": user_agent + "2382rgby",
        #         "Accept-Language": "en-US,en;q=0.5",
        #         "Accept-Encoding": "gzip, deflate, br",
        #         "Connection": "keep-alive",
        #         "Referer": "https://www.example.com/",
        #         "Cache-Control": "max-age=0",
        #     }
        # ).json()
        url = "https://seeking-alpha.p.rapidapi.com/articles/get-details"

        querystring = {"id": article_id}

        headers = {
            "X-RapidAPI-Key": 'ebc4f1b043msh8cd86a1b34ccf04p1a2452jsn26419110e8fb',
            "X-RapidAPI-Host": "seeking-alpha.p.rapidapi.com"
        }

        response = requests.request("GET", url, headers=headers, params=querystring).json()
        if response.get("data", None):
            break
        print(f"Retry {retry} {response}")
        retry -= 1
        sleep(1)
    if retry == 0:
        print(f"Skipped: {article_id}")
        continue
    sleep(1)
    if _ % 100 == 0:
        print(_, " ", article_id)
    else :
        print(article_id)
    # page = requests.get(url=i, headers={
    #     "User-Agent": user_agent,
    #     "Accept-Language": "en-US,en;q=0.5",
    #     "Accept-Encoding": "gzip, deflate, br",
    #     "Connection": "keep-alive",
    #     "Referer": "https://www.example.com/",
    #     "Cache-Control": "max-age=0",
    # }
    #                     )
    # with open("index.html", "w") as f:
    #     f.write(response.text)
    soup = BeautifulSoup(response["data"]["attributes"]["content"], 'html.parser')
    # call = soup.find('div',id = 'content-rail')
    # Testing if URL contains necessary elements
    test_list = []
    # call = soup.find("div", {"class": "ks-mH"})
    try:
        test = soup.find_all('p')    
    except:
        print("error soup_find", i)
    try: 
        ticker_list.append(soup.find('a').text.split(" ")[-1])
        full_text.append(test)
    except:
        print("error add full", i)
    for c in test:
        x = c.text.strip()
        test_list.append(x)
    if ('Operator' in test_list) and ('Company Participants' in test_list):
        # Basic Company Info Section
        # Create cc_id
        cc_id = counter
        # Find company ticker
        try:
            ticker = soup.find('a').text.split(" ")[-1]
            ticker_data.append(ticker)
        except:
            ticker_data.append('None')
        # Find company name
        try:
            company = soup.find('a').text.split(" ")[0:-1]
            company = " ".join(company)
            company_data.append(company)
        except:
            company_data.append('None')
        # Find call title
        try:
            title = soup.find('h1').text
            title_data.append(title)
        except:
            title_data.append('None')
        # Find quarter/year
        try:
            date_text = soup.find('p').text.split(') ')[1]
            quarter = date_text[0:7]
            quarter_data.append(quarter)
        except:
            quarter_data.append('None')
        # Find call date
        try:
            date_string = date_text[date_text.index('Call') + 5:].split("Call ")[0]
            date = date_string[:date_string.index(":") - 2].strip()
            date_data.append(date)
        except:
            date_data.append('None')
        # Find call time
        try:
            time = date_string[date_string.index(":") - 2:].strip()
            time_data.append(time)
        except:
            time_data.append('None')
        # Add CC ID
        comp_ccid.append(cc_id)
        # Speech and Company Participants Section
        bj = []
        indices = []
        convo = []
        # Pull speech script section
        script = soup.find_all('p')
        # pull strong tags from speech script section
        for i in script:
            if i.find('strong') is not None:
                g = "strong"
            else:
                g = 0
            bj.append(g)
        # Create list of index position of strong tags
        for j in range(len(bj)):
            if bj[j] == 'strong':
                indices.append(j)
        # Create list of text from speech script
        for k in script:
            x = k.text.strip()
            convo.append(x)
        # Create script ID & Participant ID
        script_id = counter2
        part_id = counter3
        # Create index objects to find positions of speech & participant sections
        try:
              conf_part_index = convo[convo.index('Conference Call Participants') + 1:convo.index('Operator')]
        except:
              conf_part_index = convo[convo.index('Company Participants') + 1:convo.index('Operator')]
        try:
            speech_index = indices[0:indices.index(convo.index('Question-and-Answer Session'))]
        except:
            speech_index = indices[0:indices.index(convo.index('Question-And-Answer Session'))]
        try:
            QA_index = indices[indices.index(convo.index('Question-and-Answer Session')) + 1:]
        except:
            QA_index = indices[indices.index(convo.index('Question-And-Answer Session')) + 1:]
        try:
            if article_id == 4420066 or article_id == "4420066":
                  comp_part_index = 0
            else:
                  comp_part_index = convo[convo.index('Company Participants') + 1:convo.index('Conference Call Participants')]
        except:
            comp_part_index = convo[
                              convo.index('Corporate Participants') + 1:convo.index('Conference Call Participants')]
        # Pulling info from speech section
        for i in range(2, len(speech_index) - 1):
            try:
                z = convo[speech_index[i] + 1:speech_index[i + 1]]
                speech.append(z)
            except:
                speech.append('None')
            try:
                g = convo[speech_index[i]]
                speaker.append(g)
            except:
                speaker.append('None')
            # Add CC ID & Script ID
            speech_ccid.append(cc_id)
            speech_scriptid.append(script_id)
        counter2 += 1
        script_id = counter2
        # Pulling info from Q&A section
        for i in range(0, len(QA_index) - 1):
            try:
                q = convo[QA_index[i] + 1:QA_index[i + 1]]
                QA_speech.append(q)
            except:
                QA_speech.append('None')
            try:
                f = convo[QA_index[i]]
                QA_speaker.append(f)
            except:
                QA_speaker.append('None')
            # Add CC ID & Script ID
            QA_ccid.append(cc_id)
            QA_scriptid.append(script_id)
        counter2 += 1
        script_id = counter2
        # Pulling company participants
        for i in comp_part_index:
            x = i.split('-')
            try:
                a = x[0]
                comp_parts.append(a)
            except:
                comp_parts.append('None')
            try:
                b = x[1]
                comp_pos.append(b)
            except:
                comp_pos.append('None')
            # Add CC ID & Part ID
            comppart_ccid.append(cc_id)
            comppart_partid.append(part_id)
            counter3 += 1
            part_id = counter3
        # Pulling conference participants
        for i in conf_part_index:
            x = i.split('-')
            try:
                a = x[0]
                conf_parts.append(a)
            except:
                conf_parts.append('None')
            try:
                b = x[1]
                conf_pos.append(b)
            except:
                conf_pos.append('None')
            # Add CC ID & Part ID
            confpart_ccid.append(cc_id)
            confpart_partid.append(part_id)
            counter3 += 1
            part_id = counter3
        # Add to CC ID
        counter += 1


0   4600254
4574934
4550156
4527231
4504834
4482308
4463261
4441901
4422230
4401478
4382943
4362707
4341792
4319666
4300690
4279301
4258275
4236503
4217359
4192790
4597258
4587729
4585266
4580024
error add full https://seekingalpha.com/article/4580024-abbvie-inc-2022-q4-results-earnings-call-presentation
4577016
4569124
error add full https://seekingalpha.com/article/4569124-41st-annual-j-p-morgan-healthcare-conference
4569123
4561798
4561180
4558075
4550412
4540985
4527609
4515809
4505115
4493846
4483642
4483607
error add full https://seekingalpha.com/article/4483607-abbvie-inc-2022-q4-results-earnings-call-presentation
4480867
error add full https://seekingalpha.com/article/4480867-abbvie-abbv-presents-j-p-morgan-healthcare-conference
4479162
4595034
4572315
4569066
4558527
error add full https://seekingalpha.com/article/4558527-cardiovascular-systems-inc-2023-q1-results-earnings-call-presentation
4553704
error add full https://seekingalpha.com/article/4553704-cardiovascular-systems-

4504951
error add full https://seekingalpha.com/article/4504951-amazon-com-inc-2022-q1-results-earnings-call-presentation
4504852
4489695
error add full https://seekingalpha.com/article/4489695-1life-healthcare-incs-onem-ceo-amir-dan-rubin-on-q4-2021-results-earnings-call-transcript
4484436
error add full https://seekingalpha.com/article/4484436-amazon-com-inc-2022-q4-results-earnings-call-presentation
4484212
4465268
error add full https://seekingalpha.com/article/4465268-1life-healthcare-inc-onem-ceo-amir-rubin-on-q3-2021-results-earnings-call-transcript
4463364
error add full https://seekingalpha.com/article/4463364-amazon-com-inc-2021-q3-results-earnings-call-presentation
4463311
4445208
error add full https://seekingalpha.com/article/4445208-1life-healthcare-inc-onem-ceo-amir-rubin-on-q2-2021-results-earnings-call-transcript
4442839
4442797
error add full https://seekingalpha.com/article/4442797-amazon-com-inc-2021-q2-results-earnings-call-presentation
4584133
200   4584132
error 

4505227
4496007
4493467
error add full https://seekingalpha.com/article/4493467-turning-point-therapeutics-tptx-investor-presentation-slideshow
4484518
4594202
error add full https://seekingalpha.com/article/4594202-citigroup-inc-2023-q1-results-earnings-call-presentation
4594201
4585815
4579223
4578261
4569853
4569852
error add full https://seekingalpha.com/article/4569852-citigroup-inc-2022-q4-results-earnings-call-presentation
4563050
4546664
error add full https://seekingalpha.com/article/4546664-citigroup-inc-2022-q3-results-earnings-call-presentation
4546663
4540763
4523649
4523568
error add full https://seekingalpha.com/article/4523568-citigroup-inc-2022-q2-results-earnings-call-presentation
4518623
4516333
4501550
4499423
error add full https://seekingalpha.com/article/4499423-citigroup-c-investor-presentation-slideshow
4479735
4479651
error add full https://seekingalpha.com/article/4479651-citigroup-inc-2021-q4-results-earnings-call-presentation
4474187
4597281
4597204
error a

4431850
4413524
4585746
4584599
4578786
error add full https://seekingalpha.com/article/4578786-cisco-systems-inc-2023-q2-results-earnings-call-presentation
4578785
4563107
4562717
4561230
4558596
4558589
error add full https://seekingalpha.com/article/4558589-cisco-systems-inc-2023-q1-results-earnings-call-presentation
4538242
4535341
4535329
error add full https://seekingalpha.com/article/4535329-cisco-systems-inc-2022-q4-results-earnings-call-presentation
4518396
4517315
4513929
4513048
error add full https://seekingalpha.com/article/4513048-cisco-systems-inc-2022-q3-results-earnings-call-presentation
4513027
4494666
4487756
error add full https://seekingalpha.com/article/4487756-cisco-systems-inc-2022-q2-results-earnings-call-presentation
4474131
4599367
4599312
error add full https://seekingalpha.com/article/4599312-cvs-health-corporation-2023-q1-results-earnings-call-presentation
4576392
4576391
error add full https://seekingalpha.com/article/4576391-cvs-health-corporation-2022-q

4424640
4424492
error add full https://seekingalpha.com/article/4424492-emerson-electric-co-2021-q2-results-earnings-call-presentation
4402951
4402839
error add full https://seekingalpha.com/article/4402839-emerson-electric-co-2021-q1-results-earnings-call-presentation
4384728
4599477
error add full https://seekingalpha.com/article/4599477-exelon-corporation-2023-q1-results-earnings-call-presentation
4599476
700   4578204
error add full https://seekingalpha.com/article/4578204-exelon-corporation-2022-q4-results-earnings-call-presentation
4578192
4558842
error add full https://seekingalpha.com/article/4558842-exelon-corporation-exc-investor-presentation-slideshow
4552711
4552651
error add full https://seekingalpha.com/article/4552651-exelon-corporation-2022-q3-results-earnings-call-presentation
4529242
4529087
error add full https://seekingalpha.com/article/4529087-exelon-corporation-2022-q2-results-earnings-call-presentation
4521382
error add full https://seekingalpha.com/article/45213

...

4401479
error add full https://seekingalpha.com/article/4401479-facebook-inc-fb-ceo-mark-zuckerberg-on-q4-2020-results-earnings-call-transcript
4401446
error add full https://seekingalpha.com/article/4401446-facebook-inc-2020-q4-results-earnings-call-presentation
4596435
4596296
error add full https://seekingalpha.com/article/4596296-3m-company-2023-q1-results-earnings-call-presentation
4589612
4587675
4582078
4571946
4571931
error add full https://seekingalpha.com/article/4571931-3m-company-2022-q4-results-earnings-call-presentation
4561450
4548991
4548909
error add full https://seekingalpha.com/article/4548909-3m-company-2022-q3-results-earnings-call-presentation
4541032
4525872
4525807
error add full https://seekingalpha.com/article/4525807-3m-company-2022-q2-results-earnings-call-presentation
4515774
4503593
4503551
error add full https://seekingalpha.com/article/4503551-3m-company-2022-q1-results-earnings-call-presentation
4496293
4495767
4481840
error add full https://seekingalph

4596343
4582074
4576832
4546171
4522924
4503658
4485846
4458544
4439020
4419264
4405254
4377233
4358280
4358179
4358081
error add full https://seekingalpha.com/article/4358081-pepsico-inc-2020-q2-results-earnings-call-presentation
4340588
4326305
error add full https://seekingalpha.com/article/4326305-pepsico-pep-presents-cagny-conference-2020-slideshow
4323987
4294866
4274115
4599079
4589392
error add full https://seekingalpha.com/article/4589392-pfizer-pfe-investor-presentation-slideshow
4586456
error add full https://seekingalpha.com/article/4586456-pfizer-pfe-presents-43rd-annual-cowen-healthcare-conference-slideshow
4585306
4579097
4574126
error add full https://seekingalpha.com/article/4574126-pfizer-inc-2022-q4-results-earnings-call-presentation
4574007
4568894
4559106
4552575
error add full https://seekingalpha.com/article/4552575-pfizer-pfe-investor-presentation-slideshow
4551649
4551529
error add full https://seekingalpha.com/article/4551529-pfizer-inc-2022-q3-results-earning

4407192
4407063
error add full https://seekingalpha.com/article/4407063-southern-company-2020-q4-results-earnings-call-presentation
4382819
4382702
error add full https://seekingalpha.com/article/4382702-southern-company-2020-q3-results-earnings-call-presentation
4599154
4575748
4551472
4551382
error add full https://seekingalpha.com/article/4551382-simon-property-group-inc-2022-q3-results-earnings-call-presentation
4528397
4509341
4484996
4464257
4444187
4427193
4405890
error add full https://seekingalpha.com/article/4405890-simon-property-group-inc-2020-q4-results-earnings-call-presentation
4404489
4387421
4367404
4346521
error add full https://seekingalpha.com/article/4346521-simon-property-group-inc-2020-q1-results-earnings-call-presentation
4346442
4329615
error add full https://seekingalpha.com/article/4329615-simon-property-group-spg-presents-raymond-james-institutional-investors-conference-slideshow
4321345
4321149
error add full https://seekingalpha.com/article/4321149-simon-p

4382136
4595007
4594972
error add full https://seekingalpha.com/article/4594972-u-s-bancorp-2023-q1-results-earnings-call-presentation
4585353
4572254
4572219
error add full https://seekingalpha.com/article/4572219-u-s-bancorp-2022-q4-results-earnings-call-presentation
4563054
4562774
error add full https://seekingalpha.com/article/4562774-goldman-sachs-2022-us-financial-services-conference
4546608
4546573
error add full https://seekingalpha.com/article/4546573-u-s-bancorp-2022-q3-results-earnings-call-presentation
4540403
4523636
4523570
error add full https://seekingalpha.com/article/4523570-u-s-bancorp-2022-q2-results-earnings-call-presentation
4518360
4516357
4515597
4501485
error add full https://seekingalpha.com/article/4501485-u-s-bancorp-2022-q1-results-earnings-call-presentation
4501457
4494109
4480407
4480288
error add full https://seekingalpha.com/article/4480288-u-s-bancorp-2021-q4-results-earnings-call-presentation
4596552
4587673
4585749
4584461
4584443
4572849
4561542
45

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [86]:
# else:
# continue
# Pause in between trying each call transcript
sleep(random.randint(1, 2))
#Create company info table
company_info_table_3 = pd.DataFrame({'CC ID':comp_ccid,'Tickers':ticker_data,'Company':company_data,'Call Title':title_data,'Fiscal Quarter':quarter_data,'Date':date_data,'Time':time_data})
#Create Speech table
Speech_table_3 = pd.DataFrame({'CC ID':speech_ccid,'Script ID':speech_scriptid,'Participant':speaker,'Speech':speech,'Section':'Speech'})
#Create Q&A table
QA_table_3 = pd.DataFrame({'CC ID':QA_ccid,'Script ID':QA_scriptid,'Participant':QA_speaker,'Speech':QA_speech,'Section':'Q&A'})
#Create Company participant table
Comp_part_table_3 = pd.DataFrame({'CC ID':comppart_ccid,'Part ID':comppart_partid,'Participant':comp_parts,'Position':comp_pos,'Organization':'None','Participant Type':'Internal'})
#Create Conference participant table
Conf_part_table_3 = pd.DataFrame({'CC ID':confpart_ccid,'Part ID':confpart_partid,'Participant':conf_parts,'Position':'None','Organization':conf_pos,'Participant Type':'External'})


In [87]:
Speech_table_3

,CC ID,Script ID,Participant,Speech,Section
0,0,0,Operator,"[Good day, and welcome to the Apple’s Q2 Fisca...",Speech
1,0,0,Suhasini Chandramouli,[Thank you. Good afternoon and thank you for j...,Speech
2,0,0,Tim Cook,"[Thank you, Suhasini. Good afternoon, everyone...",Speech
3,1,2,Operator,"[Good day, everyone, and welcome to the Apple ...",Speech
4,1,2,Tejas Gala,[Thank you. Speaking first today is Apple's CE...,Speech
...,...,...,...,...,...
3512,879,1758,Stephen Littleton,"[Thank you, and good morning, everyone. Welcom...",Speech
3513,879,1758,Darren Woods,"[Thank you, Stephen, and good morning, everyon...",Speech
3514,880,1760,Operator,"[Good day, everyone, and welcome to this Exxon...",Speech
3515,880,1760,Stephen Littleton,"[Thank you, and good morning, everyone. Welcom...",Speech


In [89]:
Calls_full_3 = pd.DataFrame({'Tickers':ticker_list,'Text':full_text})

In [90]:
Calls_full_3

,Tickers,Text
0,AAPL,"[[Apple (, [NASDAQ:, <a href=""https://seekinga..."
1,AAPL,"[[Apple, Inc. (, [NASDAQ:, <a href=""https://se..."
2,AAPL,"[[Apple Inc. (, [NASDAQ:, <a href=""https://see..."
3,AAPL,"[[Apple, Inc. (, [NASDAQ:, <a href=""https://se..."
4,AAPL,"[[Apple Inc. (, [NASDAQ:, <a href=""https://see..."
...,...,...
1390,XOM,"[[ExxonMobil Corporation (, [NYSE:, <a href=""h..."
1391,XOM,"[[Exxon Mobil Corporation (, [NYSE:, <a href=""..."
1392,XOM,"[[Exxon Mobil Corporation (, [NYSE:, <a href=""..."
1393,XOM,"[[Exxon Mobil Corporation (, [NYSE:, <a href=""..."


In [197]:
Speech_table = pd.read_csv("/Users/kseniayakunina/se/Диплом/speech_2.csv", header = 0)
Speech_table = pd.concat([Speech_table,Speech_table_2],ignore_index=True)
Speech_table

,Unnamed: 0.1,Unnamed: 0,CC ID,Script ID,Participant,Speech,Section
0,0.0,0.0,0,0,Operator,"[""Good day, and welcome to the Apple’s Q2 Fisc...",Speech
1,1.0,1.0,0,0,Suhasini Chandramouli,"[""Thank you. Good afternoon and thank you for ...",Speech
2,2.0,2.0,0,0,Tim Cook,"[""Thank you, Suhasini. Good afternoon, everyon...",Speech
3,3.0,3.0,1,2,Operator,"[""Good day, everyone, and welcome to the Apple...",Speech
4,4.0,4.0,1,2,Tejas Gala,"[""Thank you. Speaking first today is Apple's C...",Speech
...,...,...,...,...,...,...,...
3518,NaN,NaN,103,206,Stephen Littleton,"[Thank you. Good morning, everyone. Welcome to...",Speech
3519,NaN,NaN,103,206,Andrew Swiger,"[Thank you, Stephen., First and foremost, we h...",Speech
3520,NaN,NaN,103,206,Jack Williams,"[Thanks Andy., Moving to Slide 22. I'm going t...",Speech
3521,NaN,NaN,103,206,Andrew Swiger,"[Thanks., As Jack just highlighted, we have an...",Speech


In [88]:
Speech_table_3.to_csv("/Users/kseniayakunina/se/Диплом/speech_3.csv")
company_info_table_3.to_csv("/Users/kseniayakunina/se/Диплом/company_info_3.csv")
QA_table_3.to_csv("/Users/kseniayakunina/se/Диплом/qa_3.csv")
Comp_part_table_3.to_csv("/Users/kseniayakunina/se/Диплом/comp_part_3.csv")
Conf_part_table_3.to_csv("/Users/kseniayakunina/se/Диплом/conf_part_3.csv")

In [198]:
company_info_table = pd.read_csv("/Users/kseniayakunina/se/Диплом/company_info_2.csv", header = 0)
company_info_table = pd.concat([company_info_table,company_info_table_2],ignore_index=True)
QA_table = pd.read_csv("/Users/kseniayakunina/se/Диплом/qa_2.csv", header = 0)
QA_table = pd.concat([QA_table,QA_table_2],ignore_index=True)
Comp_part_table = pd.read_csv("/Users/kseniayakunina/se/Диплом/comp_part_2.csv", header = 0)
Comp_part_table = pd.concat([Comp_part_table,Comp_part_table_2],ignore_index=True)
Conf_part_table = pd.read_csv("/Users/kseniayakunina/se/Диплом/conf_part_2.csv", header = 0)
Conf_part_table = pd.concat([Conf_part_table,Conf_part_table_2],ignore_index=True)

In [203]:
company_info_table

,Unnamed: 0.1,Unnamed: 0,CC ID,Tickers,Company,Call Title,Fiscal Quarter,Date,Time
0,0.0,0.0,0,AAPL,NaN,NaN,Q2 2023,"Transcript May 4, 2023",5:00 PM ET
1,1.0,1.0,1,AAPL,NaN,NaN,Q1 2023,"February 2, 2023",5:00 PM ET
2,2.0,2.0,2,AAPL,NaN,NaN,Q4 2022,"October 27, 2022",5:00 PM ET
3,3.0,3.0,3,AAPL,NaN,NaN,Q3 2022,"July 28, 2022",5:00 PM ET
4,4.0,4.0,4,AAPL,NaN,NaN,Q2 2022,"April 28, 2022",5:00 PM ET
...,...,...,...,...,...,...,...,...,...
878,NaN,NaN,99,XOM,,None,Q3 2021,"October 29, 2021",9:30 AM ET
879,NaN,NaN,100,XOM,,None,Q2 2021,"July 30, 2021",9:30 AM ET
880,NaN,NaN,101,XOM,,None,Q1 2021,"April 30, 2021",9:30 AM ET
881,NaN,NaN,102,XOM,,None,Q4 2020,"February 2, 2021",9:30 AM ET


In [200]:
Calls_full = pd.concat([Calls_full,Calls_full_2],ignore_index=True)

In [93]:
Calls_full_3.to_csv("/Users/kseniayakunina/se/Диплом/calls_full_2.csv")

In [148]:
soup = BeautifulSoup(response["data"]["attributes"]["content"], 'html.parser')
    # call = soup.find('div',id = 'content-rail')
    # Testing if URL contains necessary elements
test_list = []
    # call = soup.find("div", {"class": "ks-mH"})

test = soup.find_all('p')
test

[<p>The following slide deck was published by Lockheed Martin Corporation in conjunction with their 2023 Q1 earnings call.</p>,
 <p><figure class="sa-presentation" data-id="92613"></figure></p>]

In [26]:
soup.find('a').text.split(" ")[-1]


'F'

In [34]:
date_text = soup.find('p').text.split(') ')[1]

In [35]:
date_text[5:].split(" ")[0]
date = date_string[:date_string.index(":") - 2].strip()
date

'February 24, 2021'

In [42]:
date_text[:-1]

'Deutsche Bank dbAccess Global ESG Conference March 9, 2023 9:00 AM E'

In [43]:
def get_url(symbol,url_list):
    url = 'https://seekingalpha.com/symbol/'+str(symbol)+"/earnings/transcripts"
    cookies = {
    'machine_cookie': '6773828693615',
    '_gcl_au': '1.1.381636789.1674914440',
    'sailthru_pageviews': '1',
    '_ga': 'GA1.1.1111259159.1674914440',
    '_uetsid': '25e318509f1411ed80fbc9b0dd149b2e',
    '_uetvid': '25e30b309f1411ed9a9dcfae82fa0541',
    '_pcid': '%7B%22browserId%22%3A%22ldg0t9tnlwgumpvz%22%7D',
    'sailthru_content': 'aecf9e6d8252c37a01aed4a287b07535',
    'sailthru_visitor': 'c497a19f-d402-41d6-91dd-e5d61cc288bc',
    '__tbc': '%7Bkpex%7DM3rChFzG8JXZM_HupgWq_-y2N1K1G9Mkfsg6LGacAFkwA75AfFmA_eSFgY7p3f_X',
    '__pat': '-18000000',
    '__pvi': 'eyJpZCI6InYtMjAyMy0wMS0yOC0xNy0wMC00MC05NTgtM29jUGJhY1lDcDFjU3VjcC1kZjY1ZjJmOGIwYzQwZjBkNjU1YjQ2ZTE2ZDNkOTgxMCIsImRvbWFpbiI6Ii5zZWVraW5nYWxwaGEuY29tIiwidGltZSI6MTY3NDkxNDQ0MTI4OH0%3D',
    '_pctx': '%7Bu%7DN4IgrgzgpgThIC4B2YA2qA05owMoBcBDfSREQpAeyRCwgEt8oBJAE0RXQF8g',
    'xbc': '%7Bkpex%7D6nlWeKT7BwPypoz7euuD-qLQECqI9iCQYiiY6g3738VAqrwGN-OKps1pMy85ZzOLgNOo7Gt_3AimxdckxRDiHUi012EiOzM9_mAMwSZZPRXKfd4PoXvROgx8xX2Xg8WmYJsWHAca-xPKJ6CgpGeIGT8bUt-ecF89dwMQ98fqlSQHkiPlKAHN6apBexP9CKRw',
    '_pxff_cc': 'U2FtZVNpdGU9TGF4Ow==',
    'pxcts': '27255bd8-9f14-11ed-9f30-727179506a4d',
    '_pxvid': '2724f9ff-9f14-11ed-9f30-727179506a4d',
    '_px2': 'eyJ1IjoiMjcwNWZlYTAtOWYxNC0xMWVkLWFlYTEtYzNjMmNiNWVkNzk3IiwidiI6IjI3MjRmOWZmLTlmMTQtMTFlZC05ZjMwLTcyNzE3OTUwNmE0ZCIsInQiOjE2NzQ5MTQ5NDMyNTMsImgiOiJhYjI4ZThlZmNiNTBiZjljOTdlNzEwZTFiYzQ3NTQ4ZTA1ZGJkYjEzYjBjYjdjNTY4OGYzMjc2MGI3ZjQ5NWQ0In0=',
    '_px': 'cTXpdC8Y0H61JpM1arizPUAVGZB55QTdInZ68KDFM1agjHpgQV5gCrdaRCAvcPmU/FJfKFrUeXggWgTgFb9LzQ==:1000:KIILVin/ErYrBhMs5Xux15IY6vxT+eynSC6WrbtzZ+O8vJ5lWFrNQW3EDe6ooUKpI+iZqJjS9H8tUnY19tw8pv4VY3hTbkzJbyDRKGw2q0G6uQgA7YnslwlBZ4oBtpLGdyhLd+3KjwxepV2940qksQP5xxZHrnce2Bdu8ga2sPICfQznWLf4e4pzUssoQwYGNVGU2E2U2DCi/V/qQ82nYSJOjoarL1Fqy1wnarurTRLxuu60MxKkOWn348La9gjNPqdAHDjXnrCcZOokCbUZyQ==',
    '_ga_KGRFF2R2C5': 'GS1.1.1674914439.1.0.1674914445.54.0.0',
    '_pxde': '97fada50da512a094b25973243a49b4d4fc9a46327780a32984f15a8077a076b:eyJ0aW1lc3RhbXAiOjE2NzQ5MTQ0NTIzNzksImZfa2IiOjB9',
    'LAST_VISITED_PAGE': '%7B%22pathname%22%3A%22https%3A%2F%2Fseekingalpha.com%2Fsymbol%2FAAPL%2Fearnings%2Ftranscripts%22%2C%22pageKey%22%3A%2207806ef3-0078-4fd9-9b6b-ce6212f793eb%22%7D',
}
    headers = {
    'authority': 'seekingalpha.com',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7',
    'cache-control': 'no-cache',
    # 'cookie': 'machine_cookie=6773828693615; _gcl_au=1.1.381636789.1674914440; sailthru_pageviews=1; _ga=GA1.1.1111259159.1674914440; _uetsid=25e318509f1411ed80fbc9b0dd149b2e; _uetvid=25e30b309f1411ed9a9dcfae82fa0541; _pcid=%7B%22browserId%22%3A%22ldg0t9tnlwgumpvz%22%7D; sailthru_content=aecf9e6d8252c37a01aed4a287b07535; sailthru_visitor=c497a19f-d402-41d6-91dd-e5d61cc288bc; __tbc=%7Bkpex%7DM3rChFzG8JXZM_HupgWq_-y2N1K1G9Mkfsg6LGacAFkwA75AfFmA_eSFgY7p3f_X; __pat=-18000000; __pvi=eyJpZCI6InYtMjAyMy0wMS0yOC0xNy0wMC00MC05NTgtM29jUGJhY1lDcDFjU3VjcC1kZjY1ZjJmOGIwYzQwZjBkNjU1YjQ2ZTE2ZDNkOTgxMCIsImRvbWFpbiI6Ii5zZWVraW5nYWxwaGEuY29tIiwidGltZSI6MTY3NDkxNDQ0MTI4OH0%3D; _pctx=%7Bu%7DN4IgrgzgpgThIC4B2YA2qA05owMoBcBDfSREQpAeyRCwgEt8oBJAE0RXQF8g; xbc=%7Bkpex%7D6nlWeKT7BwPypoz7euuD-qLQECqI9iCQYiiY6g3738VAqrwGN-OKps1pMy85ZzOLgNOo7Gt_3AimxdckxRDiHUi012EiOzM9_mAMwSZZPRXKfd4PoXvROgx8xX2Xg8WmYJsWHAca-xPKJ6CgpGeIGT8bUt-ecF89dwMQ98fqlSQHkiPlKAHN6apBexP9CKRw; _pxff_cc=U2FtZVNpdGU9TGF4Ow==; pxcts=27255bd8-9f14-11ed-9f30-727179506a4d; _pxvid=2724f9ff-9f14-11ed-9f30-727179506a4d; _px2=eyJ1IjoiMjcwNWZlYTAtOWYxNC0xMWVkLWFlYTEtYzNjMmNiNWVkNzk3IiwidiI6IjI3MjRmOWZmLTlmMTQtMTFlZC05ZjMwLTcyNzE3OTUwNmE0ZCIsInQiOjE2NzQ5MTQ5NDMyNTMsImgiOiJhYjI4ZThlZmNiNTBiZjljOTdlNzEwZTFiYzQ3NTQ4ZTA1ZGJkYjEzYjBjYjdjNTY4OGYzMjc2MGI3ZjQ5NWQ0In0=; _px=cTXpdC8Y0H61JpM1arizPUAVGZB55QTdInZ68KDFM1agjHpgQV5gCrdaRCAvcPmU/FJfKFrUeXggWgTgFb9LzQ==:1000:KIILVin/ErYrBhMs5Xux15IY6vxT+eynSC6WrbtzZ+O8vJ5lWFrNQW3EDe6ooUKpI+iZqJjS9H8tUnY19tw8pv4VY3hTbkzJbyDRKGw2q0G6uQgA7YnslwlBZ4oBtpLGdyhLd+3KjwxepV2940qksQP5xxZHrnce2Bdu8ga2sPICfQznWLf4e4pzUssoQwYGNVGU2E2U2DCi/V/qQ82nYSJOjoarL1Fqy1wnarurTRLxuu60MxKkOWn348La9gjNPqdAHDjXnrCcZOokCbUZyQ==; _ga_KGRFF2R2C5=GS1.1.1674914439.1.0.1674914445.54.0.0; _pxde=97fada50da512a094b25973243a49b4d4fc9a46327780a32984f15a8077a076b:eyJ0aW1lc3RhbXAiOjE2NzQ5MTQ0NTIzNzksImZfa2IiOjB9; LAST_VISITED_PAGE=%7B%22pathname%22%3A%22https%3A%2F%2Fseekingalpha.com%2Fsymbol%2FAAPL%2Fearnings%2Ftranscripts%22%2C%22pageKey%22%3A%2207806ef3-0078-4fd9-9b6b-ce6212f793eb%22%7D',
    'pragma': 'no-cache',
    'referer': url,
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}
    response = requests.get(
    'https://seekingalpha.com/api/v3/symbols/'+symbol+'/transcripts?filter[until]=undefined&id=aapl&include=author%2CprimaryTickers%2CsecondaryTickers%2Csentiments&isMounting=true&page[size]=20',
    cookies=cookies,
    headers=headers,
)
    soup = BeautifulSoup(response.text, "html.parser")
    list_2 = [re.findall(r"/article.+}}", x) for x in soup.text.split(" ") if re.findall(r"links.+", x)!= [] ]
    domen = "https://seekingalpha.com"
    for link in list_2:
        try:
            url_list.append(domen+link[0].split('"')[0])
        except:
            pass
    return url_list

In [94]:
stoks = pd.read_csv(r'/Users/kseniayakunina/se/Диплом/snp100.csv', header = 0, encoding='latin-1', delimiter = ";")
stoks

,Symbol,Name,Sector
0,AAPL,Apple,Information Technology
1,ABBV,AbbVie,Health Care
2,ABT,Abbott,Health Care
3,ACN,Accenture,Information Technology
4,ADBE,Adobe,Information Technology
...,...,...,...
96,VZ,Verizon,Communication Services
97,WBA,Walgreens Boots Alliance,Consumer Staples
98,WFC,Wells Fargo,Financials
99,WMT,Walmart,Consumer Staples


In [53]:
url_list = []
for i in range(len(stoks)):
    try:
        get_url(stoks.Symbol[i],url_list)
    except:
        print(stoks.Symbol[i])

In [54]:
url_list

['https://seekingalpha.com/article/4600254-apple-aapl-q2-2023-earnings-call-transcript',
 'https://seekingalpha.com/article/4574934-apple-inc-aapl-q1-2023-earnings-call-transcript',
 'https://seekingalpha.com/article/4550156-apple-inc-aapl-q4-2022-earnings-call-transcript',
 'https://seekingalpha.com/article/4527231-apple-inc-aapl-ceo-tim-cook-on-q3-2022-results-earnings-call-transcript',
 'https://seekingalpha.com/article/4504834-apple-inc-aapl-ceo-tim-cook-on-q2-2022-results-earnings-call-transcript',
 'https://seekingalpha.com/article/4482308-apple-inc-aapl-ceo-tim-cook-on-q1-2022-results-earnings-call-transcript',
 'https://seekingalpha.com/article/4463261-apple-inc-aapl-ceo-tim-cook-on-q4-2021-results-earnings-call-transcript',
 'https://seekingalpha.com/article/4441901-apple-inc-aapl-ceo-tim-cook-on-q3-2021-results-earnings-call-transcript',
 'https://seekingalpha.com/article/4422230-apple-inc-aapl-ceo-tim-cook-on-q2-2021-results-earnings-call-transcript',
 'https://seekingalpha.

In [133]:
len(url_list)

2000

In [57]:
url_list[-1]

'https://seekingalpha.com/article/4383411-exxon-mobil-corporation-xom-q3-2020-results-earnings-call-transcript'

In [59]:
with open(r'/Users/kseniayakunina/se/Диплом/urls_snp.txt', 'w') as fp:
    for item in url_list:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [61]:
stoks.Symbol.values

array(['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'AIG', 'AMD', 'AMGN', 'AMT',
       'AMZN', 'AVGO', 'AXP', 'BA', 'BAC', 'BK', 'BKNG', 'BLK', 'BMY',
       'BRK.B', 'C', 'CAT', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST',
       'CRM', 'CSCO', 'CVS', 'CVX', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR',
       'EXC', 'F', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL', 'GS',
       'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KO', 'LIN',
       'LLY', 'LMT', 'LOW', 'MA', 'MCD', 'MDLZ', 'MDT', 'MET', 'META',
       'MMM', 'MO', 'MRK', 'MS', 'MSFT', 'NEE', 'NFLX', 'NKE', 'NVDA',
       'ORCL', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTX', 'SBUX',
       'SCHW', 'SO', 'SPG', 'T', 'TGT', 'TMO', 'TMUS', 'TSLA', 'TXN',
       'UNH', 'UNP', 'UPS', 'USB', 'V', 'VZ', 'WBA', 'WFC', 'WMT', 'XOM'],
      dtype=object)

In [95]:
cusip = pd.read_excel("/Users/kseniayakunina/Downloads/Ксюша_CUSIP (1).xlsx", names = ["ticker","cusip"])
cusip

,ticker,cusip
0,AAPL,037833100
1,ABBV,00287Y100
2,ABT,002824100
3,ACN,G1151C101
4,ADBE,00724F101
...,...,...
95,VZ,92343V104
96,WBA,931427108
97,WFC,949746101
98,WMT,931142103


In [92]:
cusip['FirstWord']

0     00206R
1     002824
2     00287Y
3     00724F
4     007903
       ...  
95    931427
96    949746
97    G1151C
98    G5494J
99    G5960L
Name: FirstWord, Length: 100, dtype: object

In [96]:
merged_df = pd.merge(stoks, cusip, how='left', left_on='Symbol', right_on='ticker')

In [97]:
merged_df["Cusip"] = merged_df.cusip
merged_df = merged_df.drop(["ticker","cusip"], axis = 1)
merged_df

,Symbol,Name,Sector,Cusip
0,AAPL,Apple,Information Technology,037833100
1,ABBV,AbbVie,Health Care,00287Y100
2,ABT,Abbott,Health Care,002824100
3,ACN,Accenture,Information Technology,G1151C101
4,ADBE,Adobe,Information Technology,00724F101
...,...,...,...,...
96,VZ,Verizon,Communication Services,92343V104
97,WBA,Walgreens Boots Alliance,Consumer Staples,931427108
98,WFC,Wells Fargo,Financials,949746101
99,WMT,Walmart,Consumer Staples,931142103


In [11]:
merged_df.to_csv(r'/Users/kseniayakunina/se/Диплом/snp100_cusip.csv')

In [9]:
merged_df = pd.read_csv('/Users/kseniayakunina/se/Диплом/snp100_cusip.csv', header = 0, index_col = 0)

In [3]:
cusip[["cusip"]].to_csv(r'/Users/kseniayakunina/se/Диплом/cusip.txt',
                                  header=None, index=None, sep='\t', mode='a')

In [107]:
new_cusip = company_info_table[["Tickers","Date"]].copy()

In [ ]:
pd.merge(df1, df2, left_on="timestamp1", right_on="timestamp2", how="inner")

In [108]:
new_cusip = pd.merge(new_cusip, cusip,left_on="Tickers", right_on="ticker", how="left")

In [109]:
new_cusip.drop("ticker", axis = 1, inplace = True)

In [110]:
new_cusip.drop(new_cusip[new_cusip.Date  == "None"].index, axis = 0, inplace = True)

In [111]:
from dateutil.parser import parse
new_cusip["Date"] = new_cusip["Date"].apply(lambda x: parse(x) if type(x) == str else x)

In [131]:
new_cusip.drop(new_cusip[new_cusip.Tickers == "None"].index, axis = 0, inplace= True)

In [113]:
from datetime import datetime as dt
new_cusip["Date"] = new_cusip["Date"].apply(lambda x: dt.strftime(x, '%Y/%m/%d'))

In [132]:
new_cusip[["cusip_2","date"]].to_csv(r'/Users/kseniayakunina/se/Диплом/cusip_dates.txt',
                                  header=None, index=None, sep='\t')

In [98]:
str(new_cusip.cusip.iloc[0])[:-1]

'3783310'

In [23]:
merged_df["Cusip_2"] = merged_df.Cusip.apply(lambda x: str(x)[:-1])

In [116]:
new_cusip["date"] = new_cusip["Date"]

In [120]:
new_cusip[["cusip_2","date"]]

,cusip_2,date
0,03783310,2023/02/02
1,03783310,2022/10/27
2,03783310,2022/07/28
3,03783310,2022/04/28
4,03783310,2022/01/27
...,...,...
282,30161N10,2022/05/09
283,30161N10,2021/11/03
284,30161N10,2021/08/04
285,30161N10,2021/05/05


In [4]:
Speech_table = pd.read_csv("/Users/kseniayakunina/se/Диплом/speech_with_summary_short_full.csv", header = 0, index_col = 0)

In [5]:
Speech_table

,CC ID,Script ID,Participant,Speech,summary,tone
0,0,0,"['Operator', 'Tejas Gala', 'Timothy Cook', 'Lu...",['Good day everyone welcome Apple Q1 Fiscal Ye...,ss Tim Cook hell followed CFO Luca Maestri wel...,Positive
1,1,2,"['Operator', 'Tejas Gala', 'Tim Cook', 'Luca M...",['Good day welcome Apple Q4 Fiscal Year 2022 E...,ss September quarter reported record revenue 9...,Positive
2,2,4,"['Operator', 'Tejas Gala', 'Tim Cook', 'Luca M...",['Good day welcome Apple Q3 FY 2022 Earnings C...,ss Apple reporting another record June quarter...,Positive
3,3,6,"['Operator', 'Tejas Gala', 'Tim Cook', 'Luca M...",['Good day welcome Apple Q2 FY 2022 Earnings C...,ss Apple continues streak unparalleled innovat...,Positive
4,4,8,"['Operator', 'Tejas Gala', 'Tim Cook', 'Luca M...",['Good day welcome Apple Q1 FY 2022 Earnings C...,ss Apple Q1 FY 2022 Earnings Conference Call c...,Positive
...,...,...,...,...,...,...
282,282,564,"['Operator', 'Jeanne Jones', 'Chris Crane', 'J...",['Hello welcome Exelons First Quarter Earnings...,ss Exelons First Quarter Earnings Conference C...,Positive
283,283,566,"['Operator', 'Emily Duncan', 'A - Chris Crane'...",['Hello welcome Exelons Third Quarter Earnings...,ss Exelons Third Quarter Earnings Call Third q...,Neutral
284,284,568,"['Operator', 'Dan Eggers', 'Chris Crane', 'Joe...",['Hello welcome Exelon’s Second Quarter Earnin...,ss Exelon’s Second Quarter Earnings Call We ea...,Neutral
285,285,570,"['Operator', 'Dan Eggers', 'Chris Crane', 'Joe...",['Hello welcome Exelons First Quarter Earnings...,ss Exelons First Quarter Earnings Call Exelon ...,Positive


In [124]:
speech_new = pd.DataFrame()
speech_new["CC ID"] = Speech_table_3["CC ID"].unique()
speech_new["Script ID"] = Speech_table_3["Script ID"].unique()

In [125]:
a = []

for i in Speech_table_3.groupby("Script ID").Speech:
      a.append(list(i[1].values ))
b = []

for i in Speech_table_3.groupby("Script ID").Participant:
      b.append(list(i[1].values ))
speech_new["Participant"] = b
speech_new["Speech"] = a

In [6]:
company_info = pd.read_csv("/Users/kseniayakunina/se/Диплом/company_info.csv",header = 0, index_col = 0)

In [7]:
company_info.iloc[75]

CC ID                     75
Tickers                 AMGN
Company                  NaN
Call Title               NaN
Fiscal Quarter       America
Date                     NaN
Time              8:00 AM ET
Name: 75, dtype: object

In [126]:
df = pd.merge(speech_new,company_info_table_3[["CC ID","Tickers","Fiscal Quarter","Date"]], how = "left", left_on='CC ID', right_on='CC ID')

In [12]:
merged_df

,Symbol,Name,Sector,Cusip
0,AAPL,Apple,Information Technology,037833100
1,ABBV,AbbVie,Health Care,00287Y100
2,ABT,Abbott,Health Care,002824100
3,ACN,Accenture,Information Technology,G1151C101
4,ADBE,Adobe,Information Technology,00724F101
...,...,...,...,...
96,VZ,Verizon,Communication Services,92343V104
97,WBA,Walgreens Boots Alliance,Consumer Staples,931427108
98,WFC,Wells Fargo,Financials,949746101
99,WMT,Walmart,Consumer Staples,931142103


In [127]:
df

,CC ID,Script ID,Participant,Speech,Tickers,Fiscal Quarter,Date
0,0,0,"[Operator, Suhasini Chandramouli, Tim Cook]","[[Good day, and welcome to the Apple’s Q2 Fisc...",AAPL,Q2 2023,"Transcript May 4, 2023"
1,1,2,"[Operator, Tejas Gala, Timothy Cook, Luca Maes...","[[Good day, everyone, and welcome to the Apple...",AAPL,Q1 2023,"February 2, 2023"
2,2,4,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]","[[Good day, and welcome to the Apple Q4 Fiscal...",AAPL,Q4 2022,"October 27, 2022"
3,3,6,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]","[[Good day, and welcome to the Apple Q3 FY 202...",AAPL,Q3 2022,"July 28, 2022"
4,4,8,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]",[[Good day and welcome to the Apple Q2 FY 2022...,AAPL,Q2 2022,"April 28, 2022"
...,...,...,...,...,...,...,...
873,876,1752,"[Operator, Stephen Littleton, Darren Woods]","[[Good day, everyone, and welcome to this Exxo...",XOM,Q4 2021,"February 1, 2022"
874,877,1754,"[Operator, Stephen Littleton, Darren Woods]",[[Please standby. We're about to begin. Good d...,XOM,Q3 2021,"October 29, 2021"
875,878,1756,"[Operator, Stephen Littleton, Darren Woods, St...","[[Good day, everyone, and welcome to the Exxon...",XOM,Q2 2021,"July 30, 2021"
876,879,1758,"[Operator, Stephen Littleton, Darren Woods]","[[Good day, everyone, and welcome to this Exxo...",XOM,Q1 2021,"April 30, 2021"


In [128]:
df = pd.merge(df,merged_df[["Symbol","Sector","Cusip"]], how = "left", left_on='Tickers', right_on='Symbol')

In [129]:
df = df.drop("Symbol", axis = 1)
df

,CC ID,Script ID,Participant,Speech,Tickers,Fiscal Quarter,Date,Sector,Cusip
0,0,0,"[Operator, Suhasini Chandramouli, Tim Cook]","[[Good day, and welcome to the Apple’s Q2 Fisc...",AAPL,Q2 2023,"Transcript May 4, 2023",Information Technology,037833100
1,1,2,"[Operator, Tejas Gala, Timothy Cook, Luca Maes...","[[Good day, everyone, and welcome to the Apple...",AAPL,Q1 2023,"February 2, 2023",Information Technology,037833100
2,2,4,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]","[[Good day, and welcome to the Apple Q4 Fiscal...",AAPL,Q4 2022,"October 27, 2022",Information Technology,037833100
3,3,6,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]","[[Good day, and welcome to the Apple Q3 FY 202...",AAPL,Q3 2022,"July 28, 2022",Information Technology,037833100
4,4,8,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]",[[Good day and welcome to the Apple Q2 FY 2022...,AAPL,Q2 2022,"April 28, 2022",Information Technology,037833100
...,...,...,...,...,...,...,...,...,...
873,876,1752,"[Operator, Stephen Littleton, Darren Woods]","[[Good day, everyone, and welcome to this Exxo...",XOM,Q4 2021,"February 1, 2022",Energy,30231G102
874,877,1754,"[Operator, Stephen Littleton, Darren Woods]",[[Please standby. We're about to begin. Good d...,XOM,Q3 2021,"October 29, 2021",Energy,30231G102
875,878,1756,"[Operator, Stephen Littleton, Darren Woods, St...","[[Good day, everyone, and welcome to the Exxon...",XOM,Q2 2021,"July 30, 2021",Energy,30231G102
876,879,1758,"[Operator, Stephen Littleton, Darren Woods]","[[Good day, everyone, and welcome to this Exxo...",XOM,Q1 2021,"April 30, 2021",Energy,30231G102


In [130]:
df['Date'] = df['Date'].str.replace('Transcript ', '')
df.head()

,CC ID,Script ID,Participant,Speech,Tickers,Fiscal Quarter,Date,Sector,Cusip
0,0,0,"[Operator, Suhasini Chandramouli, Tim Cook]","[[Good day, and welcome to the Apple’s Q2 Fisc...",AAPL,Q2 2023,"May 4, 2023",Information Technology,037833100
1,1,2,"[Operator, Tejas Gala, Timothy Cook, Luca Maes...","[[Good day, everyone, and welcome to the Apple...",AAPL,Q1 2023,"February 2, 2023",Information Technology,037833100
2,2,4,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]","[[Good day, and welcome to the Apple Q4 Fiscal...",AAPL,Q4 2022,"October 27, 2022",Information Technology,037833100
3,3,6,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]","[[Good day, and welcome to the Apple Q3 FY 202...",AAPL,Q3 2022,"July 28, 2022",Information Technology,037833100
4,4,8,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]",[[Good day and welcome to the Apple Q2 FY 2022...,AAPL,Q2 2022,"April 28, 2022",Information Technology,037833100


In [131]:
df.drop(df[df['Date'] == "None"].index, inplace = True)

In [132]:
from datetime import datetime as dt
from dateutil.parser import parse
df["Date"] = df["Date"].apply(lambda x: parse(x) if type(x) == str else x)
#df["Date"] = df["Date"].apply(lambda x: dt.strftime(x, '%Y/%m/%d'))

In [ ]:
df["Date"] = df["Date"].apply(lambda x: dt.strftime(x, '%Y/%m/%d'))

In [28]:
car = pd.read_csv("/Users/kseniayakunina/se/Диплом/qtdct6wrxgjvdcb8o.csv", header = 0)
car.head()

,ResType,_weight_,PERMNO,CRSPDay,EventDat,_int_id_,Request_File_CUSIP,CUSIP,Name,N_ar1,...,SCAR3,CAR_WLS_Weight3,N_ar4,CAR_Window_4,SCAR4,CAR_WLS_Weight4,N_ar5,CAR_Window_5,SCAR5,CAR_WLS_Weight5
0,MAR,Value,14593,25505,22945,___0000002E,03783310,03783310,APPLE INC,2,...,-1.048036,277.019885,3,0.012932,0.691905,2862.538808,5,0.004359,0.180654,1717.523285
1,MM,Value,14593,25505,22945,___0000002E,03783310,03783310,APPLE INC,2,...,-1.579770,247.761567,3,0.007342,0.391060,2836.709522,5,-0.005790,-0.237731,1686.055234
2,MAR,Value,14593,25441,22854,___0000003E,03783310,03783310,APPLE INC,2,...,-0.171958,278.175050,3,0.018485,0.991077,2874.475519,5,0.017847,0.741179,1724.685312
3,MM,Value,14593,25441,22854,___0000003E,03783310,03783310,APPLE INC,2,...,-0.656044,248.829138,3,0.005413,0.285165,2775.123916,5,0.005872,0.240887,1682.930631
4,MAR,Value,14593,25379,22763,___0000004E,03783310,03783310,APPLE INC,2,...,-0.898432,264.486244,3,0.016672,0.871563,2733.024519,5,0.003892,0.157613,1639.814711


In [67]:
car_mar = car[car.ResType == "MAR"].reset_index()

In [65]:
car_mm = car[car.ResType == "MM"].reset_index()

In [66]:
car_mm

,index,ResType,_weight_,PERMNO,CRSPDay,EventDat,_int_id_,Request_File_CUSIP,CUSIP,Name,...,SCAR3,CAR_WLS_Weight3,N_ar4,CAR_Window_4,SCAR4,CAR_WLS_Weight4,N_ar5,CAR_Window_5,SCAR5,CAR_WLS_Weight5
0,1,MM,Value,14593,25505,22945,___0000002E,03783310,03783310,APPLE INC,...,-1.579770,247.761567,3,0.007342,0.391060,2836.709522,5,-0.005790,-0.237731,1686.055234
1,3,MM,Value,14593,25441,22854,___0000003E,03783310,03783310,APPLE INC,...,-0.656044,248.829138,3,0.005413,0.285165,2775.123916,5,0.005872,0.240887,1682.930631
2,5,MM,Value,14593,25379,22763,___0000004E,03783310,03783310,APPLE INC,...,-1.005295,235.277717,3,0.016497,0.858405,2707.554675,5,0.007294,0.291635,1598.775291
3,7,MM,Value,14593,25316,22672,___0000005E,03783310,03783310,APPLE INC,...,0.133853,190.522044,3,0.052240,2.446188,2192.700476,5,0.054539,1.968832,1303.156712
4,9,MM,Value,14593,25254,22581,___0000006E,03783310,03783310,APPLE INC,...,2.077923,124.628278,3,-0.001879,-0.071004,1428.414443,5,-0.009932,-0.289624,850.332127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,439,MM,Value,21776,25386,22774,___0000252E,30161N10,30161N10,EXELON CORP,...,-1.920822,217.277886,3,0.009555,0.472610,2446.366406,5,0.025648,0.952276,1378.559062
220,441,MM,Value,21776,25258,22587,___0000253E,30161N10,30161N10,EXELON CORP,...,0.316303,223.903122,3,-0.006824,-0.346054,2571.273313,5,-0.001325,-0.051796,1529.286971
221,443,MM,Value,21776,25194,22496,___0000254E,30161N10,30161N10,EXELON CORP,...,0.842207,172.295623,3,0.006321,0.280918,1975.160670,5,0.010740,0.368365,1176.348405
222,445,MM,Value,21776,25131,22405,___0000255E,30161N10,30161N10,EXELON CORP,...,0.636641,91.012685,3,0.001675,0.054080,1042.436839,5,-0.004343,-0.108215,620.763751


In [32]:
new_cusip.drop(new_cusip[new_cusip.Date  == "None"].index, axis = 0, inplace = True)

Index(['ResType', '_weight_', 'PERMNO', 'CRSPDay', 'EventDat', '_int_id_',
       'Request_File_CUSIP', 'CUSIP', 'Name', 'N_ar1', 'CAR_Window_1', 'SCAR1',
       'CAR_WLS_Weight1', 'N_ar2', 'CAR_Window_2', 'SCAR2', 'CAR_WLS_Weight2',
       'N_ar3', 'CAR_Window_3', 'SCAR3', 'CAR_WLS_Weight3', 'N_ar4',
       'CAR_Window_4', 'SCAR4', 'CAR_WLS_Weight4', 'N_ar5', 'CAR_Window_5',
       'SCAR5', 'CAR_WLS_Weight5'],
      dtype='object')

In [15]:
df.drop(df[df.Cusip.isna() == True].index, axis = 0, inplace = True)

In [133]:
df2 = df[df.Date.dt.year > 2000]

In [19]:
df2 = df2.reset_index()

In [51]:
df.to_csv("/Users/kseniayakunina/se/Диплом/merged_df.csv")

In [134]:
fin = pd.read_csv("/Users/kseniayakunina/Downloads/snp100_fin_data.csv",header = 0, index_col = 0)
fin

,Date,Open,High,Low,Close,Adj Close,Volume,Company
0,2020-01-02,74.059998,75.150002,73.797501,75.087502,73.449394,135480400.0,AAPL
1,2020-01-03,74.287498,75.144997,74.125000,74.357498,72.735306,146322800.0,AAPL
2,2020-01-06,73.447502,74.989998,73.187500,74.949997,73.314880,118387200.0,AAPL
3,2020-01-07,74.959999,75.224998,74.370003,74.597504,72.970085,108872000.0,AAPL
4,2020-01-08,74.290001,76.110001,74.290001,75.797501,74.143898,132079200.0,AAPL
...,...,...,...,...,...,...,...,...
75595,2022-12-23,106.949997,108.739998,106.900002,108.680000,107.850105,11539400.0,XOM
75596,2022-12-27,109.269997,110.470001,108.830002,110.190002,109.348579,11962100.0,XOM
75597,2022-12-28,109.910004,110.000000,107.900002,108.379997,107.552391,10702100.0,XOM
75598,2022-12-29,107.989998,109.720001,107.989998,109.199997,108.366135,10534000.0,XOM


In [135]:
fin["Adj Close_lag_30"] = fin.groupby("Company")["Adj Close"].shift(30)

In [136]:
fin["Adj Close_lag_30"] = fin.groupby("Company")["Adj Close_lag_30"].transform(lambda x: x.fillna(x.bfill()))

In [137]:
fin.Date = pd.to_datetime(fin.Date)

In [71]:
df3 = pd.concat([df2,car_mar], axis = 1)
df3

,index,CC ID,Script ID,Participant,Speech,summary,tone,Tickers,Fiscal Quarter,Date,...,SCAR3,CAR_WLS_Weight3,N_ar4,CAR_Window_4,SCAR4,CAR_WLS_Weight4,N_ar5,CAR_Window_5,SCAR5,CAR_WLS_Weight5
0,1,1,2,"['Operator', 'Tejas Gala', 'Tim Cook', 'Luca M...",['Good day welcome Apple Q4 Fiscal Year 2022 E...,ss September quarter reported record revenue 9...,Positive,AAPL,Q4 2022,2022-10-27,...,-1.048036,277.019885,3,0.012932,0.691905,2862.538808,5,0.004359,0.180654,1717.523285
1,2,2,4,"['Operator', 'Tejas Gala', 'Tim Cook', 'Luca M...",['Good day welcome Apple Q3 FY 2022 Earnings C...,ss Apple reporting another record June quarter...,Positive,AAPL,Q3 2022,2022-07-28,...,-0.171958,278.175050,3,0.018485,0.991077,2874.475519,5,0.017847,0.741179,1724.685312
2,3,3,6,"['Operator', 'Tejas Gala', 'Tim Cook', 'Luca M...",['Good day welcome Apple Q2 FY 2022 Earnings C...,ss Apple continues streak unparalleled innovat...,Positive,AAPL,Q2 2022,2022-04-28,...,-0.898432,264.486244,3,0.016672,0.871563,2733.024519,5,0.003892,0.157613,1639.814711
3,4,4,8,"['Operator', 'Tejas Gala', 'Tim Cook', 'Luca M...",['Good day welcome Apple Q1 FY 2022 Earnings C...,ss Apple Q1 FY 2022 Earnings Conference Call c...,Positive,AAPL,Q1 2022,2022-01-27,...,0.013651,214.148760,3,0.054823,2.578947,2212.870523,5,0.059460,2.166598,1327.722314
4,5,5,10,"['Operator', 'Tejas Gala', 'Tim Cook', 'Luca M...",['Good day welcome Apple Q4 Fiscal Year 2021 E...,ss Tim Cook A year ago I spoke atmosphere unce...,Positive,AAPL,Q4 2021,2021-10-28,...,2.078384,139.316961,3,-0.001362,-0.051671,1439.608597,5,-0.008069,-0.237147,863.765158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,282,282,564,"['Operator', 'Jeanne Jones', 'Chris Crane', 'J...",['Hello welcome Exelons First Quarter Earnings...,ss Exelons First Quarter Earnings Conference C...,Positive,EXC,Q1 2022,2022-05-09,...,-0.386461,245.513509,3,0.040320,2.030841,2536.972921,5,0.091936,3.586901,1522.183753
220,283,283,566,"['Operator', 'Emily Duncan', 'A - Chris Crane'...",['Hello welcome Exelons Third Quarter Earnings...,ss Exelons Third Quarter Earnings Call Third q...,Neutral,EXC,Q3 Earn,2021-11-03,...,0.584070,251.060115,3,-0.008498,-0.432856,2594.287855,5,-0.004386,-0.173039,1556.572713
221,284,284,568,"['Operator', 'Dan Eggers', 'Chris Crane', 'Joe...",['Hello welcome Exelon’s Second Quarter Earnin...,ss Exelon’s Second Quarter Earnings Call We ea...,Neutral,EXC,Q2 2021,2021-08-04,...,0.729591,192.703076,3,0.003532,0.157618,1991.265124,5,0.007461,0.257884,1194.759074
222,285,285,570,"['Operator', 'Dan Eggers', 'Chris Crane', 'Joe...",['Hello welcome Exelons First Quarter Earnings...,ss Exelons First Quarter Earnings Call Exelon ...,Positive,EXC,Q1 2021,2021-05-05,...,0.174606,101.687611,3,-0.004419,-0.143253,1050.771976,5,-0.011656,-0.292668,630.463185


In [138]:
df_fin = pd.merge(df2,fin[["Date","Company","Adj Close","Adj Close_lag_30","Volume"]], how = "left",left_on=['Tickers',"Date"], right_on=['Company',"Date"] )

In [80]:
df3.to_csv("/Users/kseniayakunina/se/Диплом/for_car_reg.csv")

In [219]:
ps = pd.read_excel("/Users/kseniayakunina/se/Диплом/stri3tk2jzhcpnmo.xlsx")
ps

,Historical CRSP PERMNO Link to COMPUSTAT Record,Fiscal year end,Fiscal quarter end,Date_PS,Price/Sales,CUSIP
0,10104,2018-05-31,2018-11-30,2019-01-31,4.570196,68389X10
1,10104,2018-05-31,2018-11-30,2019-02-28,4.743069,68389X10
2,10104,2018-05-31,2018-11-30,2019-03-31,4.653624,68389X10
3,10104,2018-05-31,2019-02-28,2019-04-30,4.801534,68389X10
4,10104,2018-05-31,2019-02-28,2019-05-31,4.391064,68389X10
...,...,...,...,...,...,...
4550,93436,2021-12-31,2022-06-30,2022-08-31,12.857928,88160R10
4551,93436,2021-12-31,2022-06-30,2022-09-30,12.471481,88160R10
4552,93436,2021-12-31,2022-06-30,2022-10-31,10.697598,88160R10
4553,93436,2021-12-31,2022-09-30,2022-11-30,8.212526,88160R10


In [95]:
ps["Price/Sales_lad_1"] = ps.groupby("CUSIP")["Price/Sales"].shift(1)
ps["Price/Sales_lad_1"] = ps.groupby("CUSIP")["Price/Sales_lad_1"].transform(lambda x: x.fillna(x.bfill()))

In [101]:
ps["P/S_delta"] = ps["Price/Sales"] - ps["Price/Sales_lad_1"]
ps

,Historical CRSP PERMNO Link to COMPUSTAT Record,Fiscal year end,Fiscal quarter end,Date_PS,Price/Sales,CUSIP,Price/Sales_lad_1,P/S_delta
0,10104,2018-05-31,2018-11-30,2019-01-31,4.570196,68389X10,4.570196,0.000000
1,10104,2018-05-31,2018-11-30,2019-02-28,4.743069,68389X10,4.570196,0.172872
2,10104,2018-05-31,2018-11-30,2019-03-31,4.653624,68389X10,4.743069,-0.089445
3,10104,2018-05-31,2019-02-28,2019-04-30,4.801534,68389X10,4.653624,0.147910
4,10104,2018-05-31,2019-02-28,2019-05-31,4.391064,68389X10,4.801534,-0.410469
...,...,...,...,...,...,...,...,...
4550,93436,2021-12-31,2022-06-30,2022-08-31,12.857928,88160R10,14.972031,-2.114103
4551,93436,2021-12-31,2022-06-30,2022-09-30,12.471481,88160R10,12.857928,-0.386447
4552,93436,2021-12-31,2022-06-30,2022-10-31,10.697598,88160R10,12.471481,-1.773883
4553,93436,2021-12-31,2022-09-30,2022-11-30,8.212526,88160R10,10.697598,-2.485072


In [220]:
ps.Date_PS = pd.to_datetime(ps.Date_PS)

In [224]:
df_fin['month_year'] = df_fin['Date'].dt.to_period('M')

In [233]:
df_fin_2 = df_fin_2.sort_values(by = ["Company","Date"])

In [225]:
ps['month_year'] = ps['Date_PS'].dt.to_period('M')

In [228]:
df_fin["CUSIP"] = df_fin.Cusip.apply(lambda x: str(x)[:-1])

In [231]:
df_fin_2 = pd.merge(df_fin,ps[["Price/Sales",'month_year',"CUSIP"]], how = "left",left_on=['month_year',"CUSIP"], right_on=['month_year',"CUSIP"] )

In [234]:
df_fin_2

,CC ID,Script ID,Participant,Speech,Tickers,Fiscal Quarter,Date,Sector,Cusip,Company,Adj Close,Adj Close_lag_30,Volume,Price_bool,Speech_2,Sector_encoded,month_year,CUSIP,Price/Sales
11,13,26,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]","Good day, everyone. Welcome Apple Incorporated...",AAPL,Q1 2020,2020-01-28,Information Technology,037833100,AAPL,77.689812,73.449394,162234000.0,0,"[-0.01938621, 0.019299472, 0.007511008, 0.0498...",7,2020-01,03783310,5.205189
10,12,24,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]",Good day everyone. Welcome Apple Incorporated ...,AAPL,Q2 2020,2020-04-30,Information Technology,037833100,AAPL,72.018120,60.465324,183064000.0,0,"[-0.018762516, 0.016368067, 0.009369559, 0.050...",7,2020-04,03783310,4.757222
9,11,22,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]","Good day, everyone. Welcome Apple Incorporated...",AAPL,Q3 2020,2020-07-30,Information Technology,037833100,AAPL,94.570129,86.417282,158130000.0,0,"[-0.020150261, 0.017560752, 0.009075951, 0.046...",7,2020-07,03783310,6.781509
8,10,20,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]",Good day everyone welcome Apple Inc. Fourth Qu...,AAPL,Q4 2020,2020-10-29,Information Technology,037833100,AAPL,113.582397,108.677437,146129200.0,0,"[-0.018879877, 0.014624416, 0.0058270874, 0.05...",7,2020-10,03783310,6.758331
7,9,18,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]","Good day, welcome Apple Q1 Fiscal Year 2021 Ea...",AAPL,Q1 2021,2021-01-27,Information Technology,037833100,AAPL,140.160889,120.773575,140843800.0,0,"[-0.020270405, 0.014147399, 0.008642494, 0.051...",7,2021-01,03783310,8.070077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,877,1754,"[Operator, Stephen Littleton, Darren Woods]","Please standby. We're begin. Good day, everyon...",XOM,Q3 2021,2021-10-29,Energy,30231G102,XOM,60.720501,51.951958,23966300.0,0,"[-0.021789994, 0.015131891, 0.011108226, 0.051...",3,2021-10,30231G10,1.271501
566,876,1752,"[Operator, Stephen Littleton, Darren Woods]","Good day, everyone, welcome ExxonMobil Corpora...",XOM,Q4 2021,2022-02-01,Energy,30231G102,XOM,77.152145,57.298561,61033100.0,0,"[-0.024256716, 0.015431728, 0.011496681, 0.055...",3,2022-02,30231G10,1.199884
565,875,1750,"[Operator, Jennifer Driscoll, Darren Woods]","Good day, everyone, welcome Exxon Mobil Corpor...",XOM,Q1 2022,2022-04-29,Energy,30231G102,XOM,82.273102,76.222046,34656500.0,0,"[-0.021198645, 0.012581811, 0.009405191, 0.052...",3,2022-04,30231G10,1.301948
564,874,1748,"[Operator, Jennifer Driscoll, Darren Woods]","Good day, everyone, welcome Exxon Mobil Corpor...",XOM,Q2 2022,2022-07-29,Energy,30231G102,XOM,94.503441,92.514511,29825800.0,0,"[-0.019059854, 0.015469415, 0.006225825, 0.051...",3,2022-07,30231G10,1.330794


In [248]:
df_fin_2 = df_fin_2.loc[:, ~df_fin_2.columns.isin(["Speech_2", "Sector_encoded", "month_year", "CUSIP", "Company"])]

In [249]:
df_fin_2

,CC ID,Script ID,Participant,Speech,Tickers,Fiscal Quarter,Date,Sector,Cusip,Adj Close,Adj Close_lag_30,Volume,Price_bool,Price/Sales
11,13,26,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]","Good day, everyone. Welcome Apple Incorporated...",AAPL,Q1 2020,2020-01-28,Information Technology,037833100,77.689812,73.449394,162234000.0,0,5.205189
10,12,24,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]",Good day everyone. Welcome Apple Incorporated ...,AAPL,Q2 2020,2020-04-30,Information Technology,037833100,72.018120,60.465324,183064000.0,0,4.757222
9,11,22,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]","Good day, everyone. Welcome Apple Incorporated...",AAPL,Q3 2020,2020-07-30,Information Technology,037833100,94.570129,86.417282,158130000.0,0,6.781509
8,10,20,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]",Good day everyone welcome Apple Inc. Fourth Qu...,AAPL,Q4 2020,2020-10-29,Information Technology,037833100,113.582397,108.677437,146129200.0,0,6.758331
7,9,18,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]","Good day, welcome Apple Q1 Fiscal Year 2021 Ea...",AAPL,Q1 2021,2021-01-27,Information Technology,037833100,140.160889,120.773575,140843800.0,0,8.070077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,877,1754,"[Operator, Stephen Littleton, Darren Woods]","Please standby. We're begin. Good day, everyon...",XOM,Q3 2021,2021-10-29,Energy,30231G102,60.720501,51.951958,23966300.0,0,1.271501
566,876,1752,"[Operator, Stephen Littleton, Darren Woods]","Good day, everyone, welcome ExxonMobil Corpora...",XOM,Q4 2021,2022-02-01,Energy,30231G102,77.152145,57.298561,61033100.0,0,1.199884
565,875,1750,"[Operator, Jennifer Driscoll, Darren Woods]","Good day, everyone, welcome Exxon Mobil Corpor...",XOM,Q1 2022,2022-04-29,Energy,30231G102,82.273102,76.222046,34656500.0,0,1.301948
564,874,1748,"[Operator, Jennifer Driscoll, Darren Woods]","Good day, everyone, welcome Exxon Mobil Corpor...",XOM,Q2 2022,2022-07-29,Energy,30231G102,94.503441,92.514511,29825800.0,0,1.330794


Explanatory study of dataset

In [229]:
len(df_fin.CUSIP.iloc[0]) - len(ps.CUSIP.iloc[0])

0

In [107]:
ps["Mounth"] = ps.Date_PS.dt.to_period('M')

In [108]:
df["Mounth"] = df.Date.dt.to_period('M')

In [145]:
df_ps = pd.merge(df,ps[["Price/Sales","CUSIP","Price/Sales_lad_1","P/S_delta","Mounth"]], how = "left",
               left_on=['Cusip_2',"Mounth"], right_on=['CUSIP',"Mounth"] )

In [146]:
df_ps = df_ps[(df_ps.Date.dt.year < 2023)& (df_ps.Date.dt.year > 2000)]

In [147]:
df_ps["P/S_delta_2"] = df_ps["Price/Sales"]/df_ps["Price/Sales_lad_1"]

In [26]:
def boo(x):
    if x == "Positive":
        return 1
    elif x =="Neutral":
        return -1
    else:
        return -1

In [169]:
df_ps["tone_bool"] = df_ps["tone"].apply(lambda x: boo(x)).values

In [34]:
df_fin["tone_bool"] = df_fin["tone"].apply(lambda x: boo(x)).values

In [174]:
df_ps["Price/Sales"].corr(df_ps["tone_bool"])

0.18602204495457203

In [161]:
np.cov(df_ps["P/S_delta"],df_ps["tone_bool"])

array([[0.48615143, 0.00133058],
       [0.00133058, 0.20979463]])

In [136]:
from scipy.stats import spearmanr

In [162]:
# calculate Pearson's correlation
corr, _ = spearmanr(df_ps["P/S_delta"],df_ps["tone_bool"])
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: 0.010


In [ ]:
# calculate Pearson's correlation
corr, _ = spearmanr(df_ps["P/S_delta"],df_ps["tone_bool"])
print('Pearsons correlation: %.3f' % corr)

In [ ]:
# calculate Pearson's correlation
corr, _ = spearmanr(df_ps["P/S_delta"],df_ps["tone_bool"])
print('Pearsons correlation: %.3f' % corr)

In [151]:
df_ps = df_ps.drop(df_ps[df_ps["P/S_delta_2"].isna() == True].index)

In [177]:
df_ps.tone_bool

1      1
2      1
3      1
4      1
5      1
      ..
251    1
252   -1
253   -1
254    1
255   -1
Name: tone_bool, Length: 211, dtype: int64

In [215]:
fin.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,Company,Adj Close_lag_1,Adj Close_lag_30
0,2020-01-02,74.059998,75.150002,73.797501,75.087502,73.449394,135480400.0,AAPL,73.449394,73.449394
1,2020-01-03,74.287498,75.144997,74.125000,74.357498,72.735306,146322800.0,AAPL,73.449394,73.449394
2,2020-01-06,73.447502,74.989998,73.187500,74.949997,73.314880,118387200.0,AAPL,72.735306,72.735306
3,2020-01-07,74.959999,75.224998,74.370003,74.597504,72.970085,108872000.0,AAPL,73.314880,73.314880
4,2020-01-08,74.290001,76.110001,74.290001,75.797501,74.143898,132079200.0,AAPL,72.970085,72.970085
5,2020-01-09,76.809998,77.607498,76.550003,77.407501,75.718781,170108400.0,AAPL,74.143898,74.143898
6,2020-01-10,77.650002,78.167503,77.062500,77.582497,75.889961,140644800.0,AAPL,75.718781,75.718781
7,2020-01-13,77.910004,79.267502,77.787498,79.239998,77.511299,121532000.0,AAPL,75.889961,75.889961
8,2020-01-14,79.175003,79.392502,78.042503,78.169998,76.464638,161954400.0,AAPL,77.511299,77.511299
9,2020-01-15,77.962502,78.875000,77.387497,77.834999,76.136955,121923600.0,AAPL,76.464638,76.464638


In [170]:
from scipy import stats
stats.pointbiserialr(df_ps["Price/Sales"],df_ps["tone_bool"])

PointbiserialrResult(correlation=0.18602204495457197, pvalue=0.0067330394343416535)

In [186]:
df_ps.groupby('Tickers')[["Price/Sales","tone_bool"]].apply(lambda x: stats.pointbiserialr(x["Price/Sales"],
                                                                                           x["tone_bool"])[0]).mean()

/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


-0.030474717686535726

In [28]:
df.summary = df.summary.apply(lambda x: x.replace("ss", "").replace("pad", ""))

In [44]:
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [267]:

y = df_fin["Price_bool"]

# Split the combined features and target labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)

# Initialize and train the SVM classifier
clf = SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

0.37209302325581395
0.2285714285714286


In [259]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)

accuracy_score(y_test, y_pred)

0.37209302325581395

In [260]:


# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the text data
X = vectorizer.fit_transform(df_fin.summary)

# Initialize the TfidfTransformer
tfidf_transformer = TfidfTransformer()

# Apply TF-IDF transformation to the document-term matrix
X_tfidf = tfidf_transformer.fit_transform(X)


# Convert additional features (e.g., price, date) into a sparse matrix
additional_features = np.array(df_fin[["tone_bool","Adj Close","Volume"]]) # Example data
additional_features_sparse = csr_matrix(additional_features)

# Concatenate the TF-IDF matrix with additional features
combined_features = hstack((X_tfidf, additional_features_sparse))
y = df_fin["Price_bool"]

# Split the combined features and target labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)

In [265]:
y = df_fin["Price_bool"]

# Split the combined features and target labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)

# Initialize and train the SVM classifier
clf = SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

0.37209302325581395
0.2285714285714286


In [263]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Initialize and train the Random Forest classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

0.46511627906976744
0.5490196078431373


In [268]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

# Initialize and train the Random Forest classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

0.4883720930232558
0.576923076923077


In [208]:
df2["tone_bool"] = df2["tone"].apply(lambda x: boo(x)).values

In [226]:
df_fin = pd.merge(df2,fin[["Date","Company","Adj Close","Volume","Adj Close_lag_30","Adj Close_lag_1"]], how = "left",left_on=['Tickers',"Date"], right_on=['Company',"Date"] )

In [140]:
df_fin.dropna(inplace = True)

In [39]:
df_fin = pd.merge(df_fin,merged_df[["Symbol","Sector"]], how = "left",left_on=['Tickers'], right_on=['Symbol'] )

In [141]:
df_fin.head()

,CC ID,Script ID,Participant,Speech,Tickers,Fiscal Quarter,Date,Sector,Cusip,Company,Adj Close,Adj Close_lag_30,Volume
2,2,4,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]","[[Good day, and welcome to the Apple Q4 Fiscal...",AAPL,Q4 2022,2022-10-27,Information Technology,037833100,AAPL,144.339813,151.885742,109180200.0
3,3,6,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]","[[Good day, and welcome to the Apple Q3 FY 202...",AAPL,Q3 2022,2022-07-28,Information Technology,037833100,AAPL,156.632370,132.154495,81378700.0
4,4,8,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]",[[Good day and welcome to the Apple Q2 FY 2022...,AAPL,Q2 2022,2022-04-28,Information Technology,037833100,AAPL,162.654678,158.629059,130216800.0
5,5,10,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]","[[Good day, and welcome to the Apple Q1 FY 202...",AAPL,Q1 2022,2022-01-27,Information Technology,037833100,AAPL,158.059921,173.059830,121954600.0
6,6,12,"[Operator, Tejas Gala, Tim Cook, Luca Maestri]","[[Good day, and welcome to the Apple Q4 Fiscal...",AAPL,Q4 2021,2021-10-28,Information Technology,037833100,AAPL,151.237671,147.490662,100077900.0


In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

# Convert additional features (e.g., price, date) into a sparse matrix
additional_features = np.array(df_fin[["tone_bool","Adj Close","Volume"]])  # Example data
additional_features_sparse = csr_matrix(additional_features)


In [235]:
def boo_2(x):
    if x["Adj Close_lag_30"] >  x["Adj Close"]:
        return 1
    else:
        return 0

In [245]:
def boo_3(x):
    if x["Adj Close_lag_30"] <=  x["Adj Close"]:
        return 1
    else:
        return 0

In [143]:
df_fin.to_csv("/Users/kseniayakunina/se/Диплом/calls_max.csv")

In [236]:
df_fin["Price_bool"] = df_fin[["Adj Close_lag_30", "Adj Close"]].apply(lambda x: boo_2(x), axis=1).values

In [246]:
df_fin["Price_bool_3"] = df_fin[["Adj Close_lag_30", "Adj Close"]].apply(lambda x: boo_3(x), axis=1).values

In [232]:
df_fin["Price_bool_2"] = df_fin[["Adj Close_lag_1", "Adj Close"]].apply(lambda x: boo_2(x), axis=1).values

In [223]:
from scipy import stats
stats.pointbiserialr(df_fin["Price_bool"],df_fin["tone_bool"])

PointbiserialrResult(correlation=-0.03096792132989898, pvalue=0.6523669597517802)

In [235]:
from scipy import stats
stats.pointbiserialr(df_fin["Adj Close_lag_30"],df_fin["tone_bool"])

PointbiserialrResult(correlation=-0.0499870408659098, pvalue=0.46696904841463316)

In [233]:
from scipy import stats
stats.pointbiserialr(df_fin["Price_bool_2"],df_fin["tone_bool"])

PointbiserialrResult(correlation=-0.03096792132989898, pvalue=0.6523669597517802)

In [224]:
from scipy.stats import chi2_contingency

# Assuming you have two binary variables as NumPy arrays or pandas Series


# Create a contingency table
contingency_table = [[sum(df_fin["Price_bool"].values), sum(df_fin["tone_bool"].values)],
                     [len(df_fin["Price_bool"].values) - sum(df_fin["Price_bool"].values), len(df_fin["tone_bool"].values) - sum(df_fin["tone_bool"].values)]]

# Perform the Chi-square test
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

# Print the results
print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p_value}")
print(f"Degrees of freedom: {dof}")


Chi-square statistic: 0.9350899040877412
P-value: 0.33354379997244266
Degrees of freedom: 1


In [234]:
from scipy.stats import chi2_contingency

# Assuming you have two binary variables as NumPy arrays or pandas Series


# Create a contingency table
contingency_table = [[sum(df_fin["Price_bool_2"].values), sum(df_fin["tone_bool"].values)],
                     [len(df_fin["Price_bool_2"].values) - sum(df_fin["Price_bool_2"].values), len(df_fin["tone_bool"].values) - sum(df_fin["tone_bool"].values)]]

# Perform the Chi-square test
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

# Print the results
print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p_value}")
print(f"Degrees of freedom: {dof}")

Chi-square statistic: 0.9350899040877412
P-value: 0.33354379997244266
Degrees of freedom: 1


In [236]:
df_fin.groupby('Tickers')[["Price_bool","tone_bool"]].apply(lambda x: stats.pointbiserialr(x["Price_bool"],
                                                                                           x["tone_bool"])[0]).mean()

/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


0.009122176124509824

In [251]:

from scipy import stats
stats.pointbiserialr(df_fin[df_fin["tone_bool"]==1]["Price_bool"],df_fin[df_fin["tone_bool"]==1]["tone_bool"])

/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


PointbiserialrResult(correlation=nan, pvalue=nan)

In [241]:
df_fin.groupby("tone_bool")["Price_bool"].value_counts()

tone_bool  Price_bool_2
-1         1               31
           0               24
 1         1               84
           0               75
Name: count, dtype: int64

In [239]:
''' разбить на два столбца по стантименты  ---- 
градация по балам сделать для изменения 
вытащить уверенность в вероятности разметки
по отрасли попробовать объеденить булевая
относительное изменение 

Модельку прогнозирования отдельно и потом добавить колонку тональности из саммари взять вектором 
Fast text попробовать get sentence vector попробовать для текста 
Старое исследование бейзлайн модель и основнии этого 
'''

' разбить на два столбца по стантименты \n'

Baseline

In [3]:
def metrics(y_test,y_pred):
    print("Accuracy {}".format(np.round(accuracy_score(y_test, y_pred),3)))
    print("F1 {}".format(np.round(f1_score(y_test, y_pred),3)))

Only text data without anything

In [290]:
y = df_fin["Price_bool"]

# Split the combined features and target labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialize and train the SVM classifier
clf = SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

metrics(y_test,y_pred)

Accuracy 0.488
F1 0.607


In [42]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer

minmax = RobustScaler()
norm = Normalizer()
def nor(x):
        x = pd.DataFrame(minmax.fit_transform(x), columns = x.columns)
        #x = pd.DataFrame(norm.fit_transform(x), columns = x.columns)
        return x
scaled = nor(df_fin[["Adj Close","Volume"]])

In [45]:
y = df_fin["Price_bool"]

# Split the combined features and target labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(scaled, y, test_size=0.2, random_state=42)

# Initialize and train the SVM classifier
clf = SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

metrics(y_test,y_pred)

Accuracy 0.628
F1 0.2


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import ExtraTreesClassifier
import xgboost as xgb

In [47]:
print("Only text data")
vectorizer = CountVectorizer()

# Fit and transform the text data
X = vectorizer.fit_transform(df_fin.summary)

# Initialize the TfidfTransformer
tfidf_transformer = TfidfTransformer()

# Apply TF-IDF transformation to the document-term matrix
X_tfidf = tfidf_transformer.fit_transform(X)


# Convert additional features (e.g., price, date) into a sparse matrix
additional_features = np.array(df_fin[["tone_bool","Adj Close","Volume"]]) # Example data
additional_features_sparse = csr_matrix(additional_features)

# Concatenate the TF-IDF matrix with additional features
combined_features = hstack((X_tfidf, additional_features_sparse))
y = df_fin["Price_bool"]

# Split the combined features and target labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(scaled, y, test_size=0.2, random_state=42)


# Initialize and train the Random Forest classifier
svm = SVC()
forest = RandomForestClassifier(n_estimators=15, min_samples_split=6, random_state=42)
xg = xgb.XGBClassifier(max_depth = 6, eta = 0.4,alpha = 0.3, min_child_weight = 3,max_leaves = 3)
extra = ExtraTreesClassifier(n_estimators=100, random_state=0,min_samples_split = 10, max_features = 'log2')
i = 0
model_names = ["Support vector machine","Random forest","XGBoosting","ExtraTree"]
models = [svm,forest,xg,extra]

for model in models:
    print(model_names[i])
    i+=1
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    metrics(y_test, y_pred)
    print()

Only text data
Support vector machine
Accuracy 0.581
F1 0.0

Random forest
Accuracy 0.488
F1 0.083

XGBoosting
Accuracy 0.628
F1 0.5

ExtraTree
Accuracy 0.605
F1 0.261



In [214]:
print("Only text data")
vectorizer = CountVectorizer()

# Fit and transform the text data
X = vectorizer.fit_transform(df_fin.Speech)

# Initialize the TfidfTransformer
tfidf_transformer = TfidfTransformer()

# Apply TF-IDF transformation to the document-term matrix
X_tfidf = tfidf_transformer.fit_transform(X)


# Convert additional features (e.g., price, date) into a sparse matrix
additional_features = np.array(df_fin[["Adj Close","Volume"]]) # Example data
additional_features_sparse = csr_matrix(additional_features)

# Concatenate the TF-IDF matrix with additional features
combined_features = hstack((X_tfidf, additional_features_sparse))
y = df_fin["Price_bool"]

# Split the combined features and target labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)


# Initialize and train the Random Forest classifier
svm = SVC()
forest = RandomForestClassifier(n_estimators=15, min_samples_split=6, random_state=42)
xg = xgb.XGBClassifier(max_depth = 6, eta = 0.4,alpha = 0.3, min_child_weight = 3,max_leaves = 3)
extra = ExtraTreesClassifier(n_estimators=100, random_state=0,min_samples_split = 10, max_features = 'log2')
i = 0
model_names = ["Support vector machine","Random forest","XGBoosting","ExtraTree"]
models = [svm,forest,xg,extra]

for model in models:
    print(model_names[i])
    i+=1
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    metrics(y_test, y_pred)
    print()

Only text data
Support vector machine
Accuracy 0.626
F1 0.295

Random forest
Accuracy 0.574
F1 0.38

XGBoosting
Accuracy 0.6
F1 0.343

ExtraTree
Accuracy 0.6
F1 0.179



In [216]:
print("Only fin data")


y = df_fin["Price_bool"]

# Split the combined features and target labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(scaled, y, test_size=0.2, random_state=42)


# Initialize and train the Random Forest classifier
svm = SVC()
forest = RandomForestClassifier(n_estimators=15, min_samples_split=6, random_state=42)
xg = xgb.XGBClassifier(max_depth = 6, eta = 0.4,alpha = 0.3, min_child_weight = 3,max_leaves = 3)
extra = ExtraTreesClassifier(n_estimators=100, random_state=0,min_samples_split = 10, max_features = 'log2')
i = 0
model_names = ["Support vector machine","Random forest","XGBoosting","ExtraTree"]
models = [svm,forest,xg,extra]

for model in models:
    print(model_names[i])
    i+=1
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    metrics(y_test, y_pred)
    print()

Only fin data
Support vector machine
Accuracy 0.591
F1 0.175

Random forest
Accuracy 0.513
F1 0.317

XGBoosting
Accuracy 0.47
F1 0.265

ExtraTree
Accuracy 0.574
F1 0.269



In [48]:
print("Only fin data")
vectorizer = CountVectorizer()

# Fit and transform the text data
X = vectorizer.fit_transform(df_fin.summary)

# Initialize the TfidfTransformer
tfidf_transformer = TfidfTransformer()

# Apply TF-IDF transformation to the document-term matrix
X_tfidf = tfidf_transformer.fit_transform(X)


# Convert additional features (e.g., price, date) into a sparse matrix
additional_features = np.array(df_fin[["tone_bool","Adj Close","Volume"]]) # Example data
additional_features_sparse = csr_matrix(additional_features)

# Concatenate the TF-IDF matrix with additional features
combined_features = hstack((X_tfidf, additional_features_sparse))
y = df_fin["Price_bool"]

# Split the combined features and target labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(scaled, y, test_size=0.2, random_state=42)


# Initialize and train the Random Forest classifier
svm = SVC()
forest = RandomForestClassifier(n_estimators=15, min_samples_split=6, random_state=42)
xg = xgb.XGBClassifier(max_depth = 6, eta = 0.4,alpha = 0.3, min_child_weight = 3,max_leaves = 3)
extra = ExtraTreesClassifier(n_estimators=100, random_state=0,min_samples_split = 10, max_features = 'log2')
i = 0
model_names = ["Support vector machine","Random forest","XGBoosting","ExtraTree"]
models = [svm,forest,xg,extra]

for model in models:
    print(model_names[i])
    i+=1
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    metrics(y_test, y_pred)
    print()

Only fin data
Support vector machine
Accuracy 0.628
F1 0.2

Random forest
Accuracy 0.535
F1 0.412

XGBoosting
Accuracy 0.558
F1 0.513

ExtraTree
Accuracy 0.605
F1 0.37



In [52]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd


# instantiating and transforming the 'Gender' column of the df
one_hot = OneHotEncoder()
encoded = one_hot.fit_transform(df_fin[["Sector"]])

# one_hot object has an attribute 'categories_', which stores the array
# of categories sequentially, and those categories can serve as 
# new columns in our data frame.

df_fin[one_hot.categories_[0]] = encoded.toarray()

In [53]:
one_hot.categories_[0]

array(['Communication Services', 'Consumer Discretionary',
       'Consumer Staples', 'Energy', 'Financials', 'Health Care',
       'Industrials', 'Information Technology', 'Materials',
       'Real Estate', 'Utilities'], dtype=object)

In [317]:
vectorizer = CountVectorizer()

# Fit and transform the text data
X = vectorizer.fit_transform(df_fin.summary)

# Initialize the TfidfTransformer
tfidf_transformer = TfidfTransformer()

# Apply TF-IDF transformation to the document-term matrix
X_tfidf = tfidf_transformer.fit_transform(X)


# Convert additional features (e.g., price, date) into a sparse matrix
additional_features = np.array(df_fin[["tone_bool","Adj Close","Volume"]]) # Example data
additional_features_sparse = csr_matrix(additional_features)

# Concatenate the TF-IDF matrix with additional features
combined_features = hstack((X_tfidf, additional_features_sparse))
y = df_fin["Price_bool"]

# Split the combined features and target labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)


# Initialize and train the Random Forest classifier
svm = SVC()
forest = RandomForestClassifier(n_estimators=15, min_samples_split=6, random_state=42)
xg = xgb.XGBClassifier() #(max_depth = 6, eta = 0.4,alpha = 0.3, min_child_weight = 3,max_leaves = 3)
extra = ExtraTreesClassifier() #(n_estimators=100, random_state=0,min_samples_split = 10, max_features = 'log2')
i = 0
model_names = ["Support vector machine","Random forest","XGBoosting","ExtraTree"]
models = [svm,forest,xg,extra]

for model in models:
    print(model_names[i])
    i+=1
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    metrics(y_test, y_pred)
    print()

Support vector machine
Accuracy 0.372
F1 0.229

Random forest
Accuracy 0.419
F1 0.528

XGBoosting
Accuracy 0.488
F1 0.577

ExtraTree
Accuracy 0.442
F1 0.538



In [318]:
vectorizer = CountVectorizer()

# Fit and transform the text data
X = vectorizer.fit_transform(df_fin.summary)

# Initialize the TfidfTransformer
tfidf_transformer = TfidfTransformer()

# Apply TF-IDF transformation to the document-term matrix
X_tfidf = tfidf_transformer.fit_transform(X)


# Convert additional features (e.g., price, date) into a sparse matrix
additional_features = np.array(df_fin[["Adj Close","Volume"]]) # Example data
additional_features_sparse = csr_matrix(additional_features)

# Concatenate the TF-IDF matrix with additional features
combined_features = hstack((X_tfidf, additional_features_sparse))
y = df_fin["Price_bool"]

# Split the combined features and target labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)


# Initialize and train the Random Forest classifier
svm = SVC()
forest = RandomForestClassifier()
xg = xgb.XGBClassifier() #(max_depth = 6, eta = 0.4,alpha = 0.3, min_child_weight = 3,max_leaves = 3)
extra = ExtraTreesClassifier() #(n_estimators=100, random_state=0,min_samples_split = 10, max_features = 'log2')
i = 0
model_names = ["Support vector machine","Random forest","XGBoosting","ExtraTree"]
models = [svm,forest,xg,extra]

for model in models:
    print(model_names[i])
    i+=1
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    metrics(y_test, y_pred)
    print()

Support vector machine
Accuracy 0.372
F1 0.229

Random forest
Accuracy 0.488
F1 0.56

XGBoosting
Accuracy 0.488
F1 0.577

ExtraTree
Accuracy 0.605
F1 0.691



In [314]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

vectorizer = CountVectorizer()

# Fit and transform the text data
X = vectorizer.fit_transform(df_fin.summary)

# Initialize the TfidfTransformer
tfidf_transformer = TfidfTransformer()

# Apply TF-IDF transformation to the document-term matrix
X_tfidf = tfidf_transformer.fit_transform(X)


# Convert additional features (e.g., price, date) into a sparse matrix
additional_features = np.array(df_fin[["Adj Close","Volume"]]) # Example data
additional_features_sparse = csr_matrix(additional_features)

# Concatenate the TF-IDF matrix with additional features
combined_features = hstack((X_tfidf, additional_features_sparse))
y = df_fin["Price_bool"]

# Split the combined features and target labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)


# Initialize and train the Random Forest classifier
svm = SVC()
forest = RandomForestClassifier(n_estimators=15, min_samples_split=6, random_state=42)
xg = xgb.XGBClassifier(max_depth = 10, eta = 0.4,alpha = 0.3)
extra = ExtraTreesClassifier(n_estimators=100, random_state=0,min_samples_split = 10, max_features = 'log2')
i = 0
model_names = ["Support vector machine","Random forest","XGBoosting","ExtraTree"]
models = [svm,forest,xg,extra]

for model in models:
    print(model_names[i])
    i+=1
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    metrics(y_test, y_pred)
    print()

Support vector machine
Accuracy 0.372
F1 0.229

Random forest
Accuracy 0.512
F1 0.618

XGBoosting
Accuracy 0.605
F1 0.712

ExtraTree
Accuracy 0.465
F1 0.511



In [327]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer

minmax = RobustScaler()
norm = Normalizer()
def nor(x):
        x = pd.DataFrame(minmax.fit_transform(x), columns = x.columns)
        #x = pd.DataFrame(norm.fit_transform(x), columns = x.columns)
        return x
scaled = nor(df_fin[["Adj Close","Volume"]])

vectorizer = CountVectorizer()

# Fit and transform the text data
X = vectorizer.fit_transform(df_fin.summary)

# Initialize the TfidfTransformer
tfidf_transformer = TfidfTransformer()

# Apply TF-IDF transformation to the document-term matrix
X_tfidf = tfidf_transformer.fit_transform(X)

scaled = nor(df_fin[["Adj Close","Volume"]])

# Convert additional features (e.g., price, date) into a sparse matrix
additional_features = np.array(scaled) # Example data
additional_features_sparse = csr_matrix(additional_features)

# Concatenate the TF-IDF matrix with additional features
combined_features = hstack((X_tfidf, additional_features_sparse))
y = df_fin["Price_bool"]

# Split the combined features and target labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)


# Initialize and train the Random Forest classifier
svm = SVC()
forest = RandomForestClassifier(n_estimators=15, min_samples_split=6, random_state=42)
xg = xgb.XGBClassifier(max_depth = 10, eta = 0.4,alpha = 0.3)
extra = ExtraTreesClassifier(n_estimators=100, random_state=0,min_samples_split = 10, max_features = 'log2')
i = 0
model_names = ["Support vector machine","Random forest","XGBoosting","ExtraTree"]
models = [svm,forest,xg,extra]

for model in models:
    print(model_names[i])
    i+=1
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    metrics(y_test, y_pred)
    print()

Support vector machine
Accuracy 0.372
F1 0.229

Random forest
Accuracy 0.512
F1 0.618

XGBoosting
Accuracy 0.605
F1 0.712

ExtraTree
Accuracy 0.465
F1 0.511



In [180]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
le = LabelEncoder()

# Fit and transform the categorical feature
df_fin['Sector_encoded'] = le.fit_transform(df_fin['Sector'])

In [184]:
minmax = RobustScaler()
norm = Normalizer()
def nor(x):
        x = pd.DataFrame(minmax.fit_transform(x), columns = x.columns)
        #x = pd.DataFrame(norm.fit_transform(x), columns = x.columns)
        return x
scaled = nor(df_fin[["Adj Close","Volume"]])

vectorizer = CountVectorizer()

# Fit and transform the text data
X = vectorizer.fit_transform(df_fin.Speech)

# Initialize the TfidfTransformer
tfidf_transformer = TfidfTransformer()

# Apply TF-IDF transformation to the document-term matrix
X_tfidf = tfidf_transformer.fit_transform(X)

scaled = nor(df_fin[["Adj Close","Volume"]])

# Convert additional features (e.g., price, date) into a sparse matrix
additional_features = np.array(df_fin[["Adj Close","Volume","Sector_encoded"]]) # Example data
additional_features_sparse = csr_matrix(additional_features)

# Concatenate the TF-IDF matrix with additional features
combined_features = hstack((X_tfidf, additional_features_sparse))
y = df_fin["Price_bool"]

# Split the combined features and target labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)


# Initialize and train the Random Forest classifier
svm = SVC()
forest = RandomForestClassifier(n_estimators=15, min_samples_split=6, random_state=42)
xg = xgb.XGBClassifier(max_depth = 8, eta = 0.4,alpha = 0.3)
extra = ExtraTreesClassifier(n_estimators=100, random_state=0,min_samples_split = 10, max_features = 'log2')
i = 0
model_names = ["Support vector machine","Random forest","XGBoosting","ExtraTree"]
models = [svm,forest,xg,extra]

for model in models:
    print(model_names[i])
    i+=1
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    metrics(y_test, y_pred)
    print()

Support vector machine
Accuracy 0.591
F1 0.145

Random forest
Accuracy 0.496
F1 0.194

XGBoosting
Accuracy 0.609
F1 0.416

ExtraTree
Accuracy 0.635
F1 0.222



In [202]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier


minmax = RobustScaler()
norm = Normalizer()
def nor(x):
        x = pd.DataFrame(minmax.fit_transform(x), columns = x.columns)
        #x = pd.DataFrame(norm.fit_transform(x), columns = x.columns)
        return x
scaled = nor(df_fin[["Adj Close","Volume"]])

vectorizer = CountVectorizer()

# Fit and transform the text data
X = vectorizer.fit_transform(df_fin.Speech)

# Initialize the TfidfTransformer
tfidf_transformer = TfidfTransformer()

# Apply TF-IDF transformation to the document-term matrix
X_tfidf = tfidf_transformer.fit_transform(X)


# Convert additional features (e.g., price, date) into a sparse matrix
additional_features = np.array(scaled) # Example data
additional_features_sparse = csr_matrix(additional_features)

# Concatenate the TF-IDF matrix with additional features
combined_features = hstack((X_tfidf, additional_features_sparse))
y = df_fin["Price_bool"]

# Split the combined features and target labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)


# Initialize and train the Random Forest classifier
svm = SVC()
forest = RandomForestClassifier(n_estimators=100, min_samples_split=6, random_state=42)
xg = xgb.XGBClassifier(max_depth = 8, eta = 0.4,alpha = 0.3)
extra = ExtraTreesClassifier(n_estimators=100, random_state=42,min_samples_split = 10, max_features = 'log2')
estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('svr',)
]
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
#clf = DecisionTreeClassifier(random_state=42,min_samples_split = 15)

i = 0
model_names = ["Support vector machine","Random forest","XGBoosting","ExtraTree","Staking"]
models = [svm,forest,xg,extra,clf]

for model in models:
    print(model_names[i])
    i+=1
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    metrics(y_test, y_pred)
    print()

Support vector machine
Accuracy 0.583
F1 0.172

Random forest
Accuracy 0.591
F1 0.254

XGBoosting
Accuracy 0.6
F1 0.41

ExtraTree
Accuracy 0.591
F1 0.145

Staking
Accuracy 0.609
F1 0.0



In [241]:
df_fin_2.dropna(inplace = True)

In [244]:
minmax = RobustScaler()
norm = Normalizer()
def nor(x):
        x = pd.DataFrame(minmax.fit_transform(x), columns = x.columns)
        #x = pd.DataFrame(norm.fit_transform(x), columns = x.columns)
        return x
scaled = nor(df_fin_2[["Adj Close","Volume","Price/Sales"]])

vectorizer = CountVectorizer()

# Fit and transform the text data
X = vectorizer.fit_transform(df_fin_2.Speech)

# Initialize the TfidfTransformer
tfidf_transformer = TfidfTransformer()

# Apply TF-IDF transformation to the document-term matrix
X_tfidf = tfidf_transformer.fit_transform(X)


# Convert additional features (e.g., price, date) into a sparse matrix
additional_features = np.array(scaled) # Example data
additional_features_sparse = csr_matrix(additional_features)

# Concatenate the TF-IDF matrix with additional features
combined_features = hstack((X_tfidf, additional_features_sparse))
y = df_fin_2["Price_bool"]

# Split the combined features and target labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)


# Initialize and train the Random Forest classifier
svm = SVC()
forest = RandomForestClassifier(n_estimators=100, min_samples_split=6, random_state=42)
xg = xgb.XGBClassifier(max_depth = 8, eta = 0.4,alpha = 0.3)
extra = ExtraTreesClassifier(n_estimators=100, random_state=42,min_samples_split = 10, max_features = 'log2')
estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
]
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
#clf = DecisionTreeClassifier(random_state=42,min_samples_split = 15)

i = 0
model_names = ["Support vector machine","Random forest","XGBoosting","ExtraTree","Staking"]
models = [svm,forest,xg,extra,clf]

for model in models:
    print(model_names[i])
    i+=1
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    metrics(y_test, y_pred)
    print()

Support vector machine
Accuracy 0.583
F1 0.118

Random forest
Accuracy 0.546
F1 0.169

XGBoosting
Accuracy 0.519
F1 0.297

ExtraTree
Accuracy 0.519
F1 0.071

Staking
Accuracy 0.528
F1 0.038



In [208]:
231/(340+231)

0.404553415061296

In [64]:
df_fin.to_csv("/Users/kseniayakunina/se/Диплом/df_fin.csv")

In [320]:
pip install fastText

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 886.5 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fastText: filename=fasttext-0.9.2-cp39-cp39-macosx_10_16_x86_64.whl size=302571 sha256=59ec222f2ead7527e6093be55f510759c0df69da9c22598506b8be97cc5a2329
  Stored in directory: /Users/kseniayakunina/Library/Caches/pip/wheels/64/57/bc/1741406019061d5664914b070bd3e71f6244648732bc96109e
Successfully built fastText
Note: you may need to restart the kernel to use updated packages.


In [330]:
import fasttext as ft

In [145]:
import fasttext.util

#fasttext.util.download_model('en', if_exists='ignore')
ft_en_model = fasttext.load_model('cc.en.300.bin')


In [170]:
df_fin["Speech_2"] = df_fin.Speech.apply(lambda x: ft_en_model.get_sentence_vector(x))

In [152]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kseniayakunina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [149]:

def remove_chars_from_text(text, chars):
    return "".join([ch for ch in text if ch not in chars])

def data_pre_processing(text):
      spec_chars = string.punctuation
      text = text.apply(lambda x : remove_chars_from_text(x,spec_chars)) #remove punctuations
      text = text.apply(lambda x : re.sub(r'\n'," ",x)) #remove punctuations
      text.apply(lambda x : x.lower()) #makes letter lower
      STOPWORDS = set(stopwords.words('english'))
      text  = text.apply(lambda x : ' '.join(word for word in x.split() if word not in STOPWORDS)) # delete stopwords
      return text

In [167]:
for i in range(len(df_fin)):
    single_list = []
    for lst in df_fin.Speech.iloc[i]:
        for item in lst:
            single_list.append(item)
        
        df_fin.Speech.iloc[i] = single_list

/var/folders/h2/57zxmk490nvfz7811zr7dmsc0000gp/T/ipykernel_51309/2594189345.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fin.Speech.iloc[i] = single_list
/var/folders/h2/57zxmk490nvfz7811zr7dmsc0000gp/T/ipykernel_51309/2594189345.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fin.Speech.iloc[i] = single_list
/var/folders/h2/57zxmk490nvfz7811zr7dmsc0000gp/T/ipykernel_51309/2594189345.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

/var/folders/h2/57zxmk490nvfz7811zr7dmsc0000gp/T/ipykernel_51309/2594189345.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fin.Speech.iloc[i] = single_list
/var/folders/h2/57zxmk490nvfz7811zr7dmsc0000gp/T/ipykernel_51309/2594189345.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fin.Speech.iloc[i] = single_list
/var/folders/h2/57zxmk490nvfz7811zr7dmsc0000gp/T/ipykernel_51309/2594189345.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

In [169]:
lemmatizer = WordNetLemmatizer()
df_fin.Speech = data_pre_processing(df_fin.Speech)
df_fin.Speech = df_fin.Speech.apply(lambda x : lemmatizer.lemmatize(x))

In [150]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kseniayakunina/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/kseniayakunina/nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kseniayakunina/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True